In [ ]:
import React, { useRef, useState, useCallback } from 'react';
import { Camera, X, Loader2, Sparkles } from 'lucide-react';
import { Button } from "@/components/ui/button";
import { motion, AnimatePresence } from 'framer-motion';

export default function CameraScanner({ onCapture, isProcessing }) {
  const videoRef = useRef(null);
  const canvasRef = useRef(null);
  const fileInputRef = useRef(null);
  const [isStreaming, setIsStreaming] = useState(false);
  const [stream, setStream] = useState(null);

  const startCamera = useCallback(async () => {
    try {
      const mediaStream = await navigator.mediaDevices.getUserMedia({
        video: { facingMode: 'environment', width: { ideal: 1280 }, height: { ideal: 720 } }
      });
      if (videoRef.current) {
        videoRef.current.srcObject = mediaStream;
        setStream(mediaStream);
        setIsStreaming(true);
      }
    } catch (err) {
      console.error('Camera access denied:', err);
      fileInputRef.current?.click();
    }
  }, []);

  const stopCamera = useCallback(() => {
    if (stream) {
      stream.getTracks().forEach(track => track.stop());
      setStream(null);
      setIsStreaming(false);
    }
  }, [stream]);

  const captureImage = useCallback(() => {
    if (videoRef.current && canvasRef.current) {
      const canvas = canvasRef.current;
      const video = videoRef.current;
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      const ctx = canvas.getContext('2d');
      ctx.drawImage(video, 0, 0);
      canvas.toBlob((blob) => {
        if (blob) {
          const file = new File([blob], 'food-scan.jpg', { type: 'image/jpeg' });
          onCapture(file);
          stopCamera();
        }
      }, 'image/jpeg', 0.9);
    }
  }, [onCapture, stopCamera]);

  const handleFileUpload = (e) => {
    const file = e.target.files?.[0];
    if (file) {
      onCapture(file);
    }
  };

  return (
    <div className="relative">
      <canvas ref={canvasRef} className="hidden" />
      <input
        ref={fileInputRef}
        type="file"
        accept="image/*"
        capture="environment"
        onChange={handleFileUpload}
        className="hidden"
      />

      <AnimatePresence mode="wait">
        {!isStreaming ? (
          <motion.div
            key="start"
            initial={{ opacity: 0, scale: 0.95 }}
            animate={{ opacity: 1, scale: 1 }}
            exit={{ opacity: 0, scale: 0.95 }}
            className="flex flex-col items-center gap-6"
          >
            <div className="relative">
              <div className="absolute inset-0 bg-gradient-to-br from-emerald-400 to-teal-500 rounded-full blur-2xl opacity-30 animate-pulse" />
              <motion.button
                whileHover={{ scale: 1.05 }}
                whileTap={{ scale: 0.95 }}
                onClick={startCamera}
                disabled={isProcessing}
                className="relative w-32 h-32 rounded-full bg-gradient-to-br from-emerald-500 to-teal-600 flex items-center justify-center shadow-xl shadow-emerald-500/30"
              >
                {isProcessing ? (
                  <Loader2 className="w-12 h-12 text-white animate-spin" />
                ) : (
                  <Camera className="w-12 h-12 text-white" />
                )}
              </motion.button>
            </div>
            <div className="text-center">
              <p className="text-lg font-medium text-slate-800">
                {isProcessing ? 'Analyzing your food...' : 'Tap to scan food'}
              </p>
              <p className="text-sm text-slate-500 mt-1">
                Point camera at your meal or ingredient
              </p>
            </div>
            <Button
              variant="ghost"
              onClick={() => fileInputRef.current?.click()}
              disabled={isProcessing}
              className="text-slate-600"
            >
              <Sparkles className="w-4 h-4 mr-2" />
              Upload from gallery
            </Button>
          </motion.div>
        ) : (
          <motion.div
            key="camera"
            initial={{ opacity: 0 }}
            animate={{ opacity: 1 }}
            exit={{ opacity: 0 }}
            className="relative rounded-3xl overflow-hidden bg-black"
          >
            <video
              ref={videoRef}
              autoPlay
              playsInline
              muted
              className="w-full aspect-[4/3] object-cover"
            />
            
            {/* Scanning overlay */}
            <div className="absolute inset-0 pointer-events-none">
              <div className="absolute inset-8 border-2 border-white/50 rounded-2xl">
                <div className="absolute top-0 left-0 w-8 h-8 border-t-4 border-l-4 border-emerald-400 rounded-tl-xl" />
                <div className="absolute top-0 right-0 w-8 h-8 border-t-4 border-r-4 border-emerald-400 rounded-tr-xl" />
                <div className="absolute bottom-0 left-0 w-8 h-8 border-b-4 border-l-4 border-emerald-400 rounded-bl-xl" />
                <div className="absolute bottom-0 right-0 w-8 h-8 border-b-4 border-r-4 border-emerald-400 rounded-br-xl" />
              </div>
              <motion.div 
                className="absolute left-8 right-8 h-0.5 bg-gradient-to-r from-transparent via-emerald-400 to-transparent"
                initial={{ top: '10%' }}
                animate={{ top: '90%' }}
                transition={{ duration: 2, repeat: Infinity, ease: 'linear' }}
              />
            </div>

            {/* Controls */}
            <div className="absolute bottom-6 inset-x-0 flex justify-center gap-4">
              <Button
                variant="secondary"
                size="icon"
                onClick={stopCamera}
                className="w-12 h-12 rounded-full bg-white/20 backdrop-blur-sm border-0 hover:bg-white/30"
              >
                <X className="w-6 h-6 text-white" />
              </Button>
              <motion.button
                whileTap={{ scale: 0.9 }}
                onClick={captureImage}
                className="w-16 h-16 rounded-full bg-white flex items-center justify-center shadow-lg"
              >
                <div className="w-12 h-12 rounded-full bg-gradient-to-br from-emerald-500 to-teal-600" />
              </motion.button>
            </div>
          </motion.div>
        )}
      </AnimatePresence>
    </div>
  );
}

# ---- merged ----

import React from 'react';
import { motion } from 'framer-motion';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Badge } from "@/components/ui/badge";
import { Progress } from "@/components/ui/progress";
import { 
  Flame, Scale, Apple, Droplet, Wheat, Candy, 
  ShieldAlert, Sparkles, TrendingUp, Check, AlertCircle 
} from 'lucide-react';
import { cn } from "@/lib/utils";

export default function FoodResults({ scanData, userProfile, onSave, onRescan }) {
  const getHealthColor = (score) => {
    if (score >= 8) return 'text-green-600';
    if (score >= 6) return 'text-yellow-600';
    return 'text-red-600';
  };

  const getHealthLabel = (score) => {
    if (score >= 8) return 'Excellent';
    if (score >= 6) return 'Good';
    if (score >= 4) return 'Moderate';
    return 'Poor';
  };

  const checkAllergyMatch = () => {
    if (!userProfile?.allergies?.length || !scanData.allergens?.length) return [];
    return scanData.allergens.filter(allergen => 
      userProfile.allergies.some(a => a.toLowerCase() === allergen.toLowerCase())
    );
  };

  const allergyMatches = checkAllergyMatch();

  const MacroCard = ({ icon: Icon, label, value, unit, color, target }) => (
    <Card className="border-0 shadow-sm">
      <CardContent className="p-4">
        <div className="flex items-center gap-3">
          <div className={cn("p-3 rounded-xl", color)}>
            <Icon className="w-5 h-5 text-white" />
          </div>
          <div className="flex-1">
            <p className="text-xs text-slate-500">{label}</p>
            <p className="text-xl font-bold text-slate-900">{value}{unit}</p>
            {target && (
              <Progress 
                value={(value / target) * 100} 
                className="h-1.5 mt-1"
              />
            )}
          </div>
        </div>
      </CardContent>
    </Card>
  );

  return (
    <motion.div
      initial={{ opacity: 0, y: 20 }}
      animate={{ opacity: 1, y: 0 }}
      className="space-y-4"
    >
      {/* Food Image & Name */}
      <Card className="overflow-hidden border-0 shadow-lg">
        <div className="relative h-48">
          <img 
            src={scanData.image_url} 
            alt={scanData.food_name}
            className="w-full h-full object-cover"
          />
          <div className="absolute inset-0 bg-gradient-to-t from-black/70 to-transparent" />
          <div className="absolute bottom-0 left-0 right-0 p-6">
            <h2 className="text-2xl font-bold text-white mb-2">{scanData.food_name}</h2>
            <div className="flex items-center gap-2">
              <Badge className="bg-white/20 backdrop-blur-sm text-white border-0">
                {scanData.portion_size || 'Standard portion'}
              </Badge>
              <Badge className="bg-white/20 backdrop-blur-sm text-white border-0">
                {scanData.food_category || 'Food'}
              </Badge>
            </div>
          </div>
        </div>
      </Card>

      {/* Allergy Alert */}
      {allergyMatches.length > 0 && (
        <motion.div
          initial={{ scale: 0.95 }}
          animate={{ scale: 1 }}
          className="bg-red-50 border-2 border-red-200 rounded-2xl p-4"
        >
          <div className="flex items-start gap-3">
            <div className="p-2 bg-red-100 rounded-lg">
              <AlertCircle className="w-5 h-5 text-red-600" />
            </div>
            <div>
              <p className="font-semibold text-red-900">Allergy Warning!</p>
              <p className="text-sm text-red-700 mt-1">
                Contains: {allergyMatches.join(', ')}
              </p>
            </div>
          </div>
        </motion.div>
      )}

      {/* Health Score */}
      <Card className="border-0 shadow-lg bg-gradient-to-br from-emerald-500 to-teal-600">
        <CardContent className="p-6">
          <div className="flex items-center justify-between">
            <div>
              <p className="text-emerald-100 text-sm mb-1">Health Score</p>
              <p className="text-4xl font-bold text-white">{scanData.health_score}/10</p>
              <p className="text-emerald-100 mt-1">{getHealthLabel(scanData.health_score)}</p>
            </div>
            <div className="w-24 h-24 rounded-full bg-white/20 backdrop-blur-sm flex items-center justify-center">
              <Apple className="w-12 h-12 text-white" />
            </div>
          </div>
        </CardContent>
      </Card>

      {/* Calories */}
      <Card className="border-0 shadow-lg">
        <CardContent className="p-6">
          <div className="flex items-center gap-4">
            <div className="p-4 bg-orange-100 rounded-2xl">
              <Flame className="w-8 h-8 text-orange-600" />
            </div>
            <div>
              <p className="text-sm text-slate-500">Calories</p>
              <p className="text-3xl font-bold text-slate-900">{scanData.calories}</p>
              {userProfile?.daily_calorie_target && (
                <p className="text-xs text-slate-500 mt-1">
                  {((scanData.calories / userProfile.daily_calorie_target) * 100).toFixed(1)}% of daily goal
                </p>
              )}
            </div>
          </div>
        </CardContent>
      </Card>

      {/* Macronutrients */}
      <div className="grid grid-cols-2 gap-3">
        <MacroCard 
          icon={Scale} 
          label="Protein" 
          value={scanData.protein} 
          unit="g"
          color="bg-blue-500"
          target={userProfile?.daily_protein_target}
        />
        <MacroCard 
          icon={Wheat} 
          label="Carbs" 
          value={scanData.carbs} 
          unit="g"
          color="bg-amber-500"
          target={userProfile?.daily_carbs_target}
        />
        <MacroCard 
          icon={Droplet} 
          label="Fats" 
          value={scanData.fats} 
          unit="g"
          color="bg-purple-500"
          target={userProfile?.daily_fats_target}
        />
        <MacroCard 
          icon={Candy} 
          label="Sugar" 
          value={scanData.sugar || 0} 
          unit="g"
          color="bg-pink-500"
        />
      </div>

      {/* Additional Nutrients */}
      {(scanData.fiber || scanData.sodium) && (
        <Card className="border-0 shadow-sm">
          <CardHeader>
            <CardTitle className="text-base">Additional Info</CardTitle>
          </CardHeader>
          <CardContent className="space-y-2">
            {scanData.fiber && (
              <div className="flex justify-between items-center">
                <span className="text-sm text-slate-600">Fiber</span>
                <span className="text-sm font-semibold">{scanData.fiber}g</span>
              </div>
            )}
            {scanData.sodium && (
              <div className="flex justify-between items-center">
                <span className="text-sm text-slate-600">Sodium</span>
                <span className="text-sm font-semibold">{scanData.sodium}mg</span>
              </div>
            )}
          </CardContent>
        </Card>
      )}

      {/* Allergens */}
      {scanData.allergens?.length > 0 && (
        <Card className="border-0 shadow-sm">
          <CardHeader>
            <CardTitle className="text-base flex items-center gap-2">
              <ShieldAlert className="w-4 h-4" />
              Allergens
            </CardTitle>
          </CardHeader>
          <CardContent>
            <div className="flex flex-wrap gap-2">
              {scanData.allergens.map((allergen, idx) => (
                <Badge 
                  key={idx}
                  variant="outline"
                  className={allergyMatches.includes(allergen) ? 'border-red-500 text-red-700' : ''}
                >
                  {allergen}
                </Badge>
              ))}
            </div>
          </CardContent>
        </Card>
      )}

      {/* AI Suggestions */}
      {scanData.suggestions?.length > 0 && (
        <Card className="border-0 shadow-sm">
          <CardHeader>
            <CardTitle className="text-base flex items-center gap-2">
              <Sparkles className="w-4 h-4 text-emerald-600" />
              AI Recommendations
            </CardTitle>
          </CardHeader>
          <CardContent className="space-y-3">
            {scanData.suggestions.map((suggestion, idx) => (
              <div key={idx} className="flex items-start gap-3">
                <div className="mt-0.5">
                  <TrendingUp className="w-4 h-4 text-emerald-600" />
                </div>
                <p className="text-sm text-slate-700">{suggestion}</p>
              </div>
            ))}
          </CardContent>
        </Card>
      )}

      {/* Action Buttons */}
      <div className="flex gap-3 pt-4">
        <Button 
          onClick={onSave}
          className="flex-1 h-12 bg-gradient-to-r from-emerald-600 to-teal-600 hover:from-emerald-700 hover:to-teal-700"
        >
          <Check className="w-5 h-5 mr-2" />
          Save to Diary
        </Button>
        <Button 
          onClick={onRescan}
          variant="outline"
          className="h-12 px-6"
        >
          Rescan
        </Button>
      </div>
    </motion.div>
  );
}

# ---- merged ----

import React from 'react';
import { motion } from 'framer-motion';
import { Badge } from "@/components/ui/badge";
import { Button } from "@/components/ui/button";
import { 
  Flame, Beef, Wheat, Droplets, AlertTriangle, 
  Heart, Plus, Lightbulb, ArrowRight, Sparkles 
} from 'lucide-react';

const MacroCard = ({ icon: Icon, label, value, unit, color, bgColor }) => (
  <motion.div 
    initial={{ opacity: 0, y: 20 }}
    animate={{ opacity: 1, y: 0 }}
    className={`${bgColor} rounded-2xl p-4 flex flex-col items-center gap-2`}
  >
    <div className={`w-10 h-10 rounded-xl ${color} bg-opacity-20 flex items-center justify-center`}>
      <Icon className={`w-5 h-5 ${color.replace('bg-', 'text-')}`} />
    </div>
    <span className="text-2xl font-bold text-slate-800">{value}<span className="text-sm font-normal text-slate-500">{unit}</span></span>
    <span className="text-xs text-slate-500 uppercase tracking-wide">{label}</span>
  </motion.div>
);

const HealthScoreMeter = ({ score }) => {
  const getColor = () => {
    if (score >= 8) return 'from-emerald-400 to-green-500';
    if (score >= 6) return 'from-yellow-400 to-amber-500';
    if (score >= 4) return 'from-orange-400 to-orange-500';
    return 'from-red-400 to-red-500';
  };

  return (
    <div className="relative flex items-center gap-4">
      <div className="flex-1 h-3 bg-slate-100 rounded-full overflow-hidden">
        <motion.div 
          initial={{ width: 0 }}
          animate={{ width: `${score * 10}%` }}
          transition={{ duration: 1, ease: 'easeOut' }}
          className={`h-full bg-gradient-to-r ${getColor()} rounded-full`}
        />
      </div>
      <span className="text-2xl font-bold text-slate-800">{score}<span className="text-sm text-slate-400">/10</span></span>
    </div>
  );
};

export default function ScanResult({ result, onAddToLog, onScanAgain, userAllergies = [] }) {
  const hasAllergenWarning = result.allergens?.some(a => 
    userAllergies.map(ua => ua.toLowerCase()).includes(a.toLowerCase())
  );

  return (
    <motion.div 
      initial={{ opacity: 0 }}
      animate={{ opacity: 1 }}
      className="space-y-6"
    >
      {/* Food Image & Name */}
      <div className="relative">
        {result.image_url && (
          <div className="relative h-48 rounded-3xl overflow-hidden">
            <img 
              src={result.image_url} 
              alt={result.food_name}
              className="w-full h-full object-cover"
            />
            <div className="absolute inset-0 bg-gradient-to-t from-black/60 to-transparent" />
            <div className="absolute bottom-4 left-4 right-4">
              <h2 className="text-2xl font-bold text-white">{result.food_name}</h2>
              <div className="flex items-center gap-2 mt-1">
                {result.food_category && (
                  <Badge variant="secondary" className="bg-white/20 text-white border-0">
                    {result.food_category}
                  </Badge>
                )}
                {result.portion_size && (
                  <Badge variant="secondary" className="bg-white/20 text-white border-0">
                    {result.portion_size}
                  </Badge>
                )}
              </div>
            </div>
          </div>
        )}
      </div>

      {/* Allergen Warning */}
      {hasAllergenWarning && (
        <motion.div 
          initial={{ scale: 0.95 }}
          animate={{ scale: 1 }}
          className="bg-red-50 border border-red-200 rounded-2xl p-4 flex items-start gap-3"
        >
          <AlertTriangle className="w-6 h-6 text-red-500 flex-shrink-0" />
          <div>
            <p className="font-semibold text-red-800">Allergen Alert!</p>
            <p className="text-sm text-red-600">
              Contains: {result.allergens?.filter(a => 
                userAllergies.map(ua => ua.toLowerCase()).includes(a.toLowerCase())
              ).join(', ')}
            </p>
          </div>
        </motion.div>
      )}

      {/* Calories */}
      <div className="bg-gradient-to-br from-orange-50 to-amber-50 rounded-3xl p-6">
        <div className="flex items-center justify-between">
          <div className="flex items-center gap-3">
            <div className="w-14 h-14 rounded-2xl bg-gradient-to-br from-orange-400 to-amber-500 flex items-center justify-center shadow-lg shadow-orange-200">
              <Flame className="w-7 h-7 text-white" />
            </div>
            <div>
              <p className="text-sm text-slate-500">Calories</p>
              <p className="text-3xl font-bold text-slate-800">{result.calories} <span className="text-lg font-normal text-slate-400">kcal</span></p>
            </div>
          </div>
        </div>
      </div>

      {/* Macros Grid */}
      <div className="grid grid-cols-3 gap-3">
        <MacroCard icon={Beef} label="Protein" value={result.protein || 0} unit="g" color="bg-rose-500" bgColor="bg-rose-50" />
        <MacroCard icon={Wheat} label="Carbs" value={result.carbs || 0} unit="g" color="bg-amber-500" bgColor="bg-amber-50" />
        <MacroCard icon={Droplets} label="Fats" value={result.fats || 0} unit="g" color="bg-blue-500" bgColor="bg-blue-50" />
      </div>

      {/* Additional Nutrients */}
      <div className="bg-slate-50 rounded-2xl p-4">
        <p className="text-sm font-medium text-slate-500 mb-3">Additional Nutrients</p>
        <div className="grid grid-cols-3 gap-4 text-center">
          <div>
            <p className="text-lg font-semibold text-slate-800">{result.fiber || 0}g</p>
            <p className="text-xs text-slate-500">Fiber</p>
          </div>
          <div>
            <p className="text-lg font-semibold text-slate-800">{result.sugar || 0}g</p>
            <p className="text-xs text-slate-500">Sugar</p>
          </div>
          <div>
            <p className="text-lg font-semibold text-slate-800">{result.sodium || 0}mg</p>
            <p className="text-xs text-slate-500">Sodium</p>
          </div>
        </div>
      </div>

      {/* Health Score */}
      <div className="bg-white border border-slate-200 rounded-2xl p-5">
        <div className="flex items-center gap-2 mb-3">
          <Heart className="w-5 h-5 text-emerald-500" />
          <span className="font-semibold text-slate-800">Health Score</span>
        </div>
        <HealthScoreMeter score={result.health_score || 5} />
      </div>

      {/* AI Suggestions */}
      {result.suggestions && result.suggestions.length > 0 && (
        <div className="bg-gradient-to-br from-violet-50 to-purple-50 rounded-2xl p-5">
          <div className="flex items-center gap-2 mb-4">
            <Sparkles className="w-5 h-5 text-violet-500" />
            <span className="font-semibold text-slate-800">Smart Suggestions</span>
          </div>
          <div className="space-y-3">
            {result.suggestions.map((suggestion, idx) => (
              <div key={idx} className="flex items-start gap-3 bg-white/60 rounded-xl p-3">
                <Lightbulb className="w-4 h-4 text-violet-500 mt-0.5 flex-shrink-0" />
                <p className="text-sm text-slate-700">{suggestion}</p>
              </div>
            ))}
          </div>
        </div>
      )}

      {/* Allergens List */}
      {result.allergens && result.allergens.length > 0 && (
        <div className="flex flex-wrap gap-2">
          <span className="text-sm text-slate-500">Contains:</span>
          {result.allergens.map((allergen, idx) => (
            <Badge 
              key={idx} 
              variant="outline"
              className={hasAllergenWarning && userAllergies.map(ua => ua.toLowerCase()).includes(allergen.toLowerCase()) 
                ? 'border-red-300 bg-red-50 text-red-700' 
                : 'border-slate-200'}
            >
              {allergen}
            </Badge>
          ))}
        </div>
      )}

      {/* Actions */}
      <div className="flex gap-3 pt-4">
        <Button 
          variant="outline" 
          onClick={onScanAgain}
          className="flex-1 h-14 rounded-2xl text-base"
        >
          Scan Again
        </Button>
        <Button 
          onClick={onAddToLog}
          className="flex-1 h-14 rounded-2xl text-base bg-gradient-to-r from-emerald-500 to-teal-600 hover:from-emerald-600 hover:to-teal-700"
        >
          <Plus className="w-5 h-5 mr-2" />
          Add to Log
        </Button>
      </div>
    </motion.div>
  );
}

# ---- merged ----

import React from 'react';
import { motion } from 'framer-motion';
import { Flame, Beef, Wheat, Droplets } from 'lucide-react';

const CircularProgress = ({ value, max, color, size = 120, strokeWidth = 10 }) => {
  const radius = (size - strokeWidth) / 2;
  const circumference = radius * 2 * Math.PI;
  const progress = Math.min((value / max) * 100, 100);
  const offset = circumference - (progress / 100) * circumference;

  return (
    <div className="relative" style={{ width: size, height: size }}>
      <svg width={size} height={size} className="transform -rotate-90">
        <circle
          cx={size / 2}
          cy={size / 2}
          r={radius}
          fill="none"
          stroke="currentColor"
          strokeWidth={strokeWidth}
          className="text-slate-100"
        />
        <motion.circle
          cx={size / 2}
          cy={size / 2}
          r={radius}
          fill="none"
          stroke={color}
          strokeWidth={strokeWidth}
          strokeLinecap="round"
          strokeDasharray={circumference}
          initial={{ strokeDashoffset: circumference }}
          animate={{ strokeDashoffset: offset }}
          transition={{ duration: 1, ease: 'easeOut' }}
        />
      </svg>
      <div className="absolute inset-0 flex flex-col items-center justify-center">
        <span className="text-2xl font-bold text-slate-800">{value}</span>
        <span className="text-xs text-slate-400">/ {max}</span>
      </div>
    </div>
  );
};

const MacroBar = ({ icon: Icon, label, value, max, color, bgColor }) => {
  const percentage = Math.min((value / max) * 100, 100);
  
  return (
    <div className="flex items-center gap-3">
      <div className={`w-10 h-10 rounded-xl ${bgColor} flex items-center justify-center flex-shrink-0`}>
        <Icon className={`w-5 h-5 ${color}`} />
      </div>
      <div className="flex-1 min-w-0">
        <div className="flex justify-between items-center mb-1">
          <span className="text-sm font-medium text-slate-700">{label}</span>
          <span className="text-sm text-slate-500">{value}g / {max}g</span>
        </div>
        <div className="h-2 bg-slate-100 rounded-full overflow-hidden">
          <motion.div 
            className={`h-full ${color.replace('text-', 'bg-')} rounded-full`}
            initial={{ width: 0 }}
            animate={{ width: `${percentage}%` }}
            transition={{ duration: 0.8, ease: 'easeOut' }}
          />
        </div>
      </div>
    </div>
  );
};

export default function DailyProgress({ todayScans = [], targets }) {
  const totals = (todayScans || []).reduce((acc, scan) => ({
    calories: acc.calories + (scan.calories || 0),
    protein: acc.protein + (scan.protein || 0),
    carbs: acc.carbs + (scan.carbs || 0),
    fats: acc.fats + (scan.fats || 0),
  }), { calories: 0, protein: 0, carbs: 0, fats: 0 });

  const calorieTarget = targets?.daily_calorie_target || 2000;
  const proteinTarget = targets?.daily_protein_target || 150;
  const carbsTarget = targets?.daily_carbs_target || 250;
  const fatsTarget = targets?.daily_fats_target || 65;

  return (
    <div className="bg-white rounded-3xl p-6 shadow-sm border border-slate-100">
      <h3 className="text-lg font-semibold text-slate-800 mb-6">Today's Progress</h3>
      
      <div className="flex flex-col md:flex-row items-center gap-8">
        {/* Calories Circle */}
        <div className="flex flex-col items-center">
          <div className="relative">
            <CircularProgress 
              value={Math.round(totals.calories)} 
              max={calorieTarget} 
              color="#f97316"
              size={140}
              strokeWidth={12}
            />
            <div className="absolute -bottom-1 left-1/2 -translate-x-1/2">
              <div className="bg-gradient-to-r from-orange-400 to-amber-500 text-white text-xs font-medium px-3 py-1 rounded-full flex items-center gap-1">
                <Flame className="w-3 h-3" />
                kcal
              </div>
            </div>
          </div>
          <p className="text-sm text-slate-500 mt-4">
            {calorieTarget - Math.round(totals.calories) > 0 
              ? `${calorieTarget - Math.round(totals.calories)} kcal remaining`
              : 'Goal reached!'
            }
          </p>
        </div>

        {/* Macros */}
        <div className="flex-1 w-full space-y-4">
          <MacroBar 
            icon={Beef} 
            label="Protein" 
            value={Math.round(totals.protein)} 
            max={proteinTarget}
            color="text-rose-500"
            bgColor="bg-rose-50"
          />
          <MacroBar 
            icon={Wheat} 
            label="Carbs" 
            value={Math.round(totals.carbs)} 
            max={carbsTarget}
            color="text-amber-500"
            bgColor="bg-amber-50"
          />
          <MacroBar 
            icon={Droplets} 
            label="Fats" 
            value={Math.round(totals.fats)} 
            max={fatsTarget}
            color="text-blue-500"
            bgColor="bg-blue-50"
          />
        </div>
      </div>
    </div>
  );
}

# ---- merged ----

import React from 'react';
import { motion } from 'framer-motion';
import { format } from 'date-fns';
import { Clock, Flame, ChevronRight, Utensils } from 'lucide-react';
import { Badge } from "@/components/ui/badge";

const mealTypeConfig = {
  breakfast: { label: 'Breakfast', icon: '🌅', color: 'bg-amber-100 text-amber-700' },
  lunch: { label: 'Lunch', icon: '☀️', color: 'bg-emerald-100 text-emerald-700' },
  dinner: { label: 'Dinner', icon: '🌙', color: 'bg-indigo-100 text-indigo-700' },
  snack: { label: 'Snack', icon: '🍎', color: 'bg-pink-100 text-pink-700' },
};

const MealCard = ({ scan, onClick }) => {
  const config = mealTypeConfig[scan.meal_type] || mealTypeConfig.snack;
  
  return (
    <motion.div 
      initial={{ opacity: 0, x: -20 }}
      animate={{ opacity: 1, x: 0 }}
      whileHover={{ scale: 1.02 }}
      onClick={onClick}
      className="bg-white rounded-2xl p-4 border border-slate-100 shadow-sm cursor-pointer flex items-center gap-4 group"
    >
      {scan.image_url ? (
        <img 
          src={scan.image_url} 
          alt={scan.food_name}
          className="w-16 h-16 rounded-xl object-cover flex-shrink-0"
        />
      ) : (
        <div className="w-16 h-16 rounded-xl bg-slate-100 flex items-center justify-center flex-shrink-0">
          <Utensils className="w-6 h-6 text-slate-400" />
        </div>
      )}
      
      <div className="flex-1 min-w-0">
        <div className="flex items-center gap-2 mb-1">
          <Badge className={`${config.color} border-0 text-xs`}>
            {config.icon} {config.label}
          </Badge>
        </div>
        <h4 className="font-semibold text-slate-800 truncate">{scan.food_name}</h4>
        <div className="flex items-center gap-3 mt-1 text-sm text-slate-500">
          <span className="flex items-center gap-1">
            <Flame className="w-3.5 h-3.5 text-orange-400" />
            {scan.calories} kcal
          </span>
          <span className="flex items-center gap-1">
            <Clock className="w-3.5 h-3.5" />
            {format(new Date(scan.created_date), 'h:mm a')}
          </span>
        </div>
      </div>
      
      <ChevronRight className="w-5 h-5 text-slate-300 group-hover:text-slate-400 transition-colors" />
    </motion.div>
  );
};

export default function MealHistory({ scans = [], onSelectMeal }) {
  const groupedByDate = (scans || []).reduce((acc, scan) => {
    const date = scan.scan_date || format(new Date(scan.created_date), 'yyyy-MM-dd');
    if (!acc[date]) acc[date] = [];
    acc[date].push(scan);
    return acc;
  }, {});

  const sortedDates = Object.keys(groupedByDate).sort((a, b) => new Date(b) - new Date(a));

  if (!scans || scans.length === 0) {
    return (
      <div className="bg-slate-50 rounded-3xl p-8 text-center">
        <div className="w-16 h-16 bg-slate-100 rounded-2xl flex items-center justify-center mx-auto mb-4">
          <Utensils className="w-8 h-8 text-slate-400" />
        </div>
        <h3 className="text-lg font-semibold text-slate-800 mb-2">No meals logged yet</h3>
        <p className="text-slate-500 text-sm">Scan your first meal to start tracking</p>
      </div>
    );
  }

  return (
    <div className="space-y-6">
      {sortedDates.map(date => {
        const isToday = date === format(new Date(), 'yyyy-MM-dd');
        const dateLabel = isToday ? 'Today' : format(new Date(date), 'EEEE, MMM d');
        const dayScans = groupedByDate[date];
        const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);

        return (
          <div key={date}>
            <div className="flex items-center justify-between mb-3">
              <h3 className="text-sm font-medium text-slate-500">{dateLabel}</h3>
              <span className="text-sm text-slate-400">{totalCalories} kcal total</span>
            </div>
            <div className="space-y-3">
              {dayScans.map(scan => (
                <MealCard 
                  key={scan.id} 
                  scan={scan} 
                  onClick={() => onSelectMeal?.(scan)}
                />
              ))}
            </div>
          </div>
        );
      })}
    </div>
  );
}

# ---- merged ----

import React from 'react';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Badge } from "@/components/ui/badge";
import { Clock, Flame, TrendingUp } from 'lucide-react';
import { motion } from 'framer-motion';
import { format } from 'date-fns';

export default function RecentScans({ scans, onScanClick }) {
  const recentScans = scans.slice(0, 5);

  const getMealIcon = (mealType) => {
    const icons = {
      breakfast: '🌅',
      lunch: '🌞',
      dinner: '🌙',
      snack: '🍎'
    };
    return icons[mealType] || '🍽️';
  };

  const getHealthColor = (score) => {
    if (score >= 8) return 'bg-green-100 text-green-800 border-green-200';
    if (score >= 6) return 'bg-yellow-100 text-yellow-800 border-yellow-200';
    return 'bg-red-100 text-red-800 border-red-200';
  };

  return (
    <Card className="border-0 shadow-lg">
      <CardHeader>
        <CardTitle className="flex items-center gap-2">
          <div className="p-2 bg-blue-100 rounded-lg">
            <Clock className="w-5 h-5 text-blue-600" />
          </div>
          Recent Scans
        </CardTitle>
      </CardHeader>
      <CardContent className="space-y-3">
        {recentScans.length === 0 ? (
          <div className="text-center py-8 text-slate-500">
            <p>No scans yet</p>
            <p className="text-sm mt-1">Start scanning your meals!</p>
          </div>
        ) : (
          recentScans.map((scan, index) => (
            <motion.div
              key={scan.id}
              initial={{ opacity: 0, y: 10 }}
              animate={{ opacity: 1, y: 0 }}
              transition={{ delay: index * 0.1 }}
              onClick={() => onScanClick?.(scan)}
              className="flex items-center gap-4 p-4 rounded-xl bg-slate-50 hover:bg-slate-100 cursor-pointer transition-colors"
            >
              <div className="relative w-16 h-16 rounded-lg overflow-hidden flex-shrink-0">
                {scan.image_url ? (
                  <img 
                    src={scan.image_url} 
                    alt={scan.food_name}
                    className="w-full h-full object-cover"
                  />
                ) : (
                  <div className="w-full h-full bg-slate-200 flex items-center justify-center text-2xl">
                    {getMealIcon(scan.meal_type)}
                  </div>
                )}
              </div>
              
              <div className="flex-1 min-w-0">
                <p className="font-semibold text-slate-900 truncate">{scan.food_name}</p>
                <div className="flex items-center gap-2 mt-1">
                  <Badge variant="outline" className="text-xs">
                    <Flame className="w-3 h-3 mr-1" />
                    {scan.calories} cal
                  </Badge>
                  {scan.health_score && (
                    <Badge className={getHealthColor(scan.health_score)}>
                      {scan.health_score}/10
                    </Badge>
                  )}
                </div>
              </div>

              <div className="text-right text-xs text-slate-500">
                {scan.scan_date && format(new Date(scan.scan_date), 'MMM d')}
              </div>
            </motion.div>
          ))
        )}
      </CardContent>
    </Card>
  );
}

# ---- merged ----

import React from 'react';
import { motion } from 'framer-motion';
import { format, subDays, startOfDay, isSameDay } from 'date-fns';
import { AreaChart, Area, XAxis, YAxis, Tooltip, ResponsiveContainer } from 'recharts';
import { TrendingUp, TrendingDown, Minus } from 'lucide-react';

export default function WeeklyChart({ scans = [], targets }) {
  const calorieTarget = targets?.daily_calorie_target || 2000;
  
  // Get last 7 days data
  const last7Days = Array.from({ length: 7 }, (_, i) => {
    const date = startOfDay(subDays(new Date(), 6 - i));
    const dayScans = (scans || []).filter(scan => 
      isSameDay(new Date(scan.created_date), date)
    );
    const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);
    const totalProtein = dayScans.reduce((sum, s) => sum + (s.protein || 0), 0);
    
    return {
      date: format(date, 'EEE'),
      fullDate: format(date, 'MMM d'),
      calories: totalCalories,
      protein: totalProtein,
      target: calorieTarget,
      mealsCount: dayScans.length
    };
  });

  const avgCalories = Math.round(
    last7Days.reduce((sum, d) => sum + d.calories, 0) / 7
  );
  
  const trend = last7Days[6].calories - last7Days[0].calories;
  const TrendIcon = trend > 100 ? TrendingUp : trend < -100 ? TrendingDown : Minus;
  const trendColor = Math.abs(avgCalories - calorieTarget) < 200 ? 'text-emerald-500' : 'text-amber-500';

  const CustomTooltip = ({ active, payload, label }) => {
    if (active && payload && payload.length) {
      return (
        <div className="bg-white rounded-xl p-3 shadow-lg border border-slate-100">
          <p className="text-sm font-medium text-slate-800 mb-1">{payload[0]?.payload?.fullDate}</p>
          <div className="space-y-1">
            <p className="text-sm text-slate-600">
              <span className="font-semibold text-orange-500">{payload[0]?.value}</span> kcal
            </p>
            <p className="text-xs text-slate-400">
              {payload[0]?.payload?.mealsCount} meals logged
            </p>
          </div>
        </div>
      );
    }
    return null;
  };

  return (
    <motion.div 
      initial={{ opacity: 0, y: 20 }}
      animate={{ opacity: 1, y: 0 }}
      className="bg-white rounded-3xl p-6 shadow-sm border border-slate-100"
    >
      <div className="flex items-start justify-between mb-6">
        <div>
          <h3 className="text-lg font-semibold text-slate-800">Weekly Overview</h3>
          <p className="text-sm text-slate-500 mt-1">Your calorie intake trend</p>
        </div>
        <div className="text-right">
          <div className="flex items-center gap-1 justify-end">
            <TrendIcon className={`w-4 h-4 ${trendColor}`} />
            <span className="text-sm font-medium text-slate-600">Avg</span>
          </div>
          <p className="text-xl font-bold text-slate-800">{avgCalories} <span className="text-sm font-normal text-slate-400">kcal/day</span></p>
        </div>
      </div>

      <div className="h-48">
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={last7Days} margin={{ top: 10, right: 10, left: -20, bottom: 0 }}>
            <defs>
              <linearGradient id="calorieGradient" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor="#f97316" stopOpacity={0.3}/>
                <stop offset="95%" stopColor="#f97316" stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              axisLine={false}
              tickLine={false}
              tick={{ fill: '#94a3b8', fontSize: 12 }}
            />
            <YAxis 
              axisLine={false}
              tickLine={false}
              tick={{ fill: '#94a3b8', fontSize: 12 }}
            />
            <Tooltip content={<CustomTooltip />} />
            <Area
              type="monotone"
              dataKey="calories"
              stroke="#f97316"
              strokeWidth={3}
              fill="url(#calorieGradient)"
            />
          </AreaChart>
        </ResponsiveContainer>
      </div>

      {/* Quick Stats */}
      <div className="grid grid-cols-3 gap-4 mt-6 pt-6 border-t border-slate-100">
        <div className="text-center">
          <p className="text-2xl font-bold text-slate-800">
            {last7Days.reduce((sum, d) => sum + d.mealsCount, 0)}
          </p>
          <p className="text-xs text-slate-500">Meals Logged</p>
        </div>
        <div className="text-center">
          <p className="text-2xl font-bold text-emerald-500">
            {last7Days.filter(d => Math.abs(d.calories - calorieTarget) < 200).length}
          </p>
          <p className="text-xs text-slate-500">Days On Target</p>
        </div>
        <div className="text-center">
          <p className="text-2xl font-bold text-slate-800">
            {Math.round(last7Days.reduce((sum, d) => sum + d.protein, 0) / 7)}g
          </p>
          <p className="text-xs text-slate-500">Avg Protein</p>
        </div>
      </div>
    </motion.div>
  );
}

# ---- merged ----

import React from 'react';
import { motion } from 'framer-motion';
import { format, subDays, startOfDay, isSameDay } from 'date-fns';
import { AreaChart, Area, XAxis, YAxis, Tooltip, ResponsiveContainer } from 'recharts';
import { TrendingUp, TrendingDown, Minus } from 'lucide-react';

export default function WeeklyChart({ scans = [], targets }) {
  const calorieTarget = targets?.daily_calorie_target || 2000;
  
  // Get last 7 days data
  const last7Days = Array.from({ length: 7 }, (_, i) => {
    const date = startOfDay(subDays(new Date(), 6 - i));
    const dayScans = (scans || []).filter(scan => 
      isSameDay(new Date(scan.created_date), date)
    );
    const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);
    const totalProtein = dayScans.reduce((sum, s) => sum + (s.protein || 0), 0);
    
    return {
      date: format(date, 'EEE'),
      fullDate: format(date, 'MMM d'),
      calories: totalCalories,
      protein: totalProtein,
      target: calorieTarget,
      mealsCount: dayScans.length
    };
  });

  const avgCalories = Math.round(
    last7Days.reduce((sum, d) => sum + d.calories, 0) / 7
  );
  
  const trend = last7Days[6].calories - last7Days[0].calories;
  const TrendIcon = trend > 100 ? TrendingUp : trend < -100 ? TrendingDown : Minus;
  const trendColor = Math.abs(avgCalories - calorieTarget) < 200 ? 'text-emerald-500' : 'text-amber-500';

  const CustomTooltip = ({ active, payload, label }) => {
    if (active && payload && payload.length) {
      return (
        <div className="bg-white rounded-xl p-3 shadow-lg border border-slate-100">
          <p className="text-sm font-medium text-slate-800 mb-1">{payload[0]?.payload?.fullDate}</p>
          <div className="space-y-1">
            <p className="text-sm text-slate-600">
              <span className="font-semibold text-orange-500">{payload[0]?.value}</span> kcal
            </p>
            <p className="text-xs text-slate-400">
              {payload[0]?.payload?.mealsCount} meals logged
            </p>
          </div>
        </div>
      );
    }
    return null;
  };

  return (
    <motion.div 
      initial={{ opacity: 0, y: 20 }}
      animate={{ opacity: 1, y: 0 }}
      className="bg-white rounded-3xl p-6 shadow-sm border border-slate-100"
    >
      <div className="flex items-start justify-between mb-6">
        <div>
          <h3 className="text-lg font-semibold text-slate-800">Weekly Overview</h3>
          <p className="text-sm text-slate-500 mt-1">Your calorie intake trend</p>
        </div>
        <div className="text-right">
          <div className="flex items-center gap-1 justify-end">
            <TrendIcon className={`w-4 h-4 ${trendColor}`} />
            <span className="text-sm font-medium text-slate-600">Avg</span>
          </div>
          <p className="text-xl font-bold text-slate-800">{avgCalories} <span className="text-sm font-normal text-slate-400">kcal/day</span></p>
        </div>
      </div>

      <div className="h-48">
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={last7Days} margin={{ top: 10, right: 10, left: -20, bottom: 0 }}>
            <defs>
              <linearGradient id="calorieGradient" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor="#f97316" stopOpacity={0.3}/>
                <stop offset="95%" stopColor="#f97316" stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              axisLine={false}
              tickLine={false}
              tick={{ fill: '#94a3b8', fontSize: 12 }}
            />
            <YAxis 
              axisLine={false}
              tickLine={false}
              tick={{ fill: '#94a3b8', fontSize: 12 }}
            />
            <Tooltip content={<CustomTooltip />} />
            <Area
              type="monotone"
              dataKey="calories"
              stroke="#f97316"
              strokeWidth={3}
              fill="url(#calorieGradient)"
            />
          </AreaChart>
        </ResponsiveContainer>
      </div>

      {/* Quick Stats */}
      <div className="grid grid-cols-3 gap-4 mt-6 pt-6 border-t border-slate-100">
        <div className="text-center">
          <p className="text-2xl font-bold text-slate-800">
            {last7Days.reduce((sum, d) => sum + d.mealsCount, 0)}
          </p>
          <p className="text-xs text-slate-500">Meals Logged</p>
        </div>
        <div className="text-center">
          <p className="text-2xl font-bold text-emerald-500">
            {last7Days.filter(d => Math.abs(d.calories - calorieTarget) < 200).length}
          </p>
          <p className="text-xs text-slate-500">Days On Target</p>
        </div>
        <div className="text-center">
          <p className="text-2xl font-bold text-slate-800">
            {Math.round(last7Days.reduce((sum, d) => sum + d.protein, 0) / 7)}g
          </p>
          <p className="text-xs text-slate-500">Avg Protein</p>
        </div>
      </div>
    </motion.div>
  );
}

# ---- merged ----

import React, { useState, useEffect } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery } from '@tanstack/react-query';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Badge } from "@/components/ui/badge";
import { ArrowLeft, TrendingUp, Target, Award, Calendar } from 'lucide-react';
import { Link } from 'react-router-dom';
import { createPageUrl } from '../utils';
import { BarChart, Bar, LineChart, Line, PieChart, Pie, Cell, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ResponsiveContainer } from 'recharts';
import { format, subDays, startOfWeek, endOfWeek } from 'date-fns';

export default function Analytics() {
  const [user, setUser] = useState(null);
  const [userProfile, setUserProfile] = useState(null);
  const [timeRange, setTimeRange] = useState('week'); // week, month

  useEffect(() => {
    const fetchUser = async () => {
      try {
        const currentUser = await base44.auth.me();
        setUser(currentUser);
        const profiles = await base44.entities.UserProfile.filter({ created_by: currentUser.email });
        if (profiles.length > 0) setUserProfile(profiles[0]);
      } catch (err) {
        console.error(err);
      }
    };
    fetchUser();
  }, []);

  const { data: scans = [] } = useQuery({
    queryKey: ['scans', user?.email],
    queryFn: () => base44.entities.FoodScan.filter({ created_by: user.email }, '-scan_date', 200),
    enabled: !!user,
    initialData: []
  });

  // Calculate weekly trends
  const getWeeklyData = () => {
    const daysOfWeek = [];
    for (let i = 6; i >= 0; i--) {
      const date = format(subDays(new Date(), i), 'yyyy-MM-dd');
      const dayScans = scans.filter(s => s.scan_date === date);
      const totalCalories = dayScans.reduce((sum, s) => sum + (s.calories || 0), 0);
      
      daysOfWeek.push({
        date: format(subDays(new Date(), i), 'EEE'),
        calories: Math.round(totalCalories),
        target: userProfile?.daily_calorie_target || 2000
      });
    }
    return daysOfWeek;
  };

  // Macro distribution
  const getMacroDistribution = () => {
    const totals = scans.reduce((acc, scan) => ({
      protein: acc.protein + (scan.protein || 0),
      carbs: acc.carbs + (scan.carbs || 0),
      fats: acc.fats + (scan.fats || 0)
    }), { protein: 0, carbs: 0, fats: 0 });

    return [
      { name: 'Protein', value: Math.round(totals.protein), color: '#3b82f6' },
      { name: 'Carbs', value: Math.round(totals.carbs), color: '#f59e0b' },
      { name: 'Fats', value: Math.round(totals.fats), color: '#a855f7' }
    ];
  };

  // Health score trend
  const getHealthScoreTrend = () => {
    return getWeeklyData().map((day, i) => {
      const date = format(subDays(new Date(), 6 - i), 'yyyy-MM-dd');
      const dayScans = scans.filter(s => s.scan_date === date);
      const avgScore = dayScans.length > 0 
        ? dayScans.reduce((sum, s) => sum + (s.health_score || 0), 0) / dayScans.length
        : 0;
      
      return {
        date: day.date,
        score: Math.round(avgScore * 10) / 10
      };
    });
  };

  // Stats
  const getTotalScans = () => scans.length;
  const getAvgHealthScore = () => {
    if (scans.length === 0) return 0;
    const total = scans.reduce((sum, s) => sum + (s.health_score || 0), 0);
    return (total / scans.length).toFixed(1);
  };
  const getWeeklyAvgCalories = () => {
    const weekData = getWeeklyData();
    const total = weekData.reduce((sum, d) => sum + d.calories, 0);
    return Math.round(total / 7);
  };

  const weeklyData = getWeeklyData();
  const macroData = getMacroDistribution();
  const healthTrend = getHealthScoreTrend();

  if (!user || !userProfile) {
    return <div className="min-h-screen flex items-center justify-center">Loading...</div>;
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-emerald-50 via-teal-50 to-cyan-50">
      {/* Header */}
      <div className="bg-white/80 backdrop-blur-sm border-b border-slate-200 sticky top-0 z-10">
        <div className="max-w-4xl mx-auto px-6 py-4 flex items-center gap-4">
          <Link to={createPageUrl('Home')}>
            <Button variant="ghost" size="icon">
              <ArrowLeft className="w-5 h-5" />
            </Button>
          </Link>
          <div>
            <h1 className="text-xl font-bold text-slate-900">Analytics</h1>
            <p className="text-sm text-slate-600">Track your nutrition journey</p>
          </div>
        </div>
      </div>

      <div className="max-w-4xl mx-auto px-6 py-8 space-y-6">
        {/* Quick Stats */}
        <div className="grid grid-cols-3 gap-4">
          <Card className="border-0 shadow-lg">
            <CardContent className="p-6 text-center">
              <div className="p-3 bg-blue-100 rounded-xl w-fit mx-auto mb-3">
                <Calendar className="w-6 h-6 text-blue-600" />
              </div>
              <p className="text-3xl font-bold text-slate-900">{getTotalScans()}</p>
              <p className="text-sm text-slate-600 mt-1">Total Scans</p>
            </CardContent>
          </Card>

          <Card className="border-0 shadow-lg">
            <CardContent className="p-6 text-center">
              <div className="p-3 bg-green-100 rounded-xl w-fit mx-auto mb-3">
                <Award className="w-6 h-6 text-green-600" />
              </div>
              <p className="text-3xl font-bold text-slate-900">{getAvgHealthScore()}</p>
              <p className="text-sm text-slate-600 mt-1">Avg Health Score</p>
            </CardContent>
          </Card>

          <Card className="border-0 shadow-lg">
            <CardContent className="p-6 text-center">
              <div className="p-3 bg-orange-100 rounded-xl w-fit mx-auto mb-3">
                <Target className="w-6 h-6 text-orange-600" />
              </div>
              <p className="text-3xl font-bold text-slate-900">{getWeeklyAvgCalories()}</p>
              <p className="text-sm text-slate-600 mt-1">Avg Daily Cal</p>
            </CardContent>
          </Card>
        </div>

        {/* Calorie Trend */}
        <Card className="border-0 shadow-lg">
          <CardHeader>
            <CardTitle className="flex items-center gap-2">
              <TrendingUp className="w-5 h-5 text-emerald-600" />
              Weekly Calorie Intake
            </CardTitle>
          </CardHeader>
          <CardContent>
            <ResponsiveContainer width="100%" height={250}>
              <BarChart data={weeklyData}>
                <CartesianGrid strokeDasharray="3 3" stroke="#e2e8f0" />
                <XAxis dataKey="date" stroke="#64748b" />
                <YAxis stroke="#64748b" />
                <Tooltip 
                  contentStyle={{ backgroundColor: '#fff', border: '1px solid #e2e8f0', borderRadius: '8px' }}
                />
                <Bar dataKey="calories" fill="#10b981" radius={[8, 8, 0, 0]} />
                <Bar dataKey="target" fill="#e2e8f0" radius={[8, 8, 0, 0]} />
              </BarChart>
            </ResponsiveContainer>
          </CardContent>
        </Card>

        {/* Macro Distribution */}
        <Card className="border-0 shadow-lg">
          <CardHeader>
            <CardTitle>Macronutrient Distribution</CardTitle>
          </CardHeader>
          <CardContent>
            <div className="flex items-center gap-8">
              <ResponsiveContainer width="50%" height={200}>
                <PieChart>
                  <Pie
                    data={macroData}
                    cx="50%"
                    cy="50%"
                    innerRadius={60}
                    outerRadius={80}
                    paddingAngle={5}
                    dataKey="value"
                  >
                    {macroData.map((entry, index) => (
                      <Cell key={`cell-${index}`} fill={entry.color} />
                    ))}
                  </Pie>
                  <Tooltip />
                </PieChart>
              </ResponsiveContainer>
              <div className="flex-1 space-y-3">
                {macroData.map((macro, idx) => (
                  <div key={idx} className="flex items-center justify-between">
                    <div className="flex items-center gap-3">
                      <div className="w-4 h-4 rounded" style={{ backgroundColor: macro.color }} />
                      <span className="text-sm font-medium text-slate-700">{macro.name}</span>
                    </div>
                    <span className="text-sm font-semibold text-slate-900">{macro.value}g</span>
                  </div>
                ))}
              </div>
            </div>
          </CardContent>
        </Card>

        {/* Health Score Trend */}
        <Card className="border-0 shadow-lg">
          <CardHeader>
            <CardTitle>Health Score Trend</CardTitle>
          </CardHeader>
          <CardContent>
            <ResponsiveContainer width="100%" height={200}>
              <LineChart data={healthTrend}>
                <CartesianGrid strokeDasharray="3 3" stroke="#e2e8f0" />
                <XAxis dataKey="date" stroke="#64748b" />
                <YAxis domain={[0, 10]} stroke="#64748b" />
                <Tooltip 
                  contentStyle={{ backgroundColor: '#fff', border: '1px solid #e2e8f0', borderRadius: '8px' }}
                />
                <Line 
                  type="monotone" 
                  dataKey="score" 
                  stroke="#10b981" 
                  strokeWidth={3}
                  dot={{ fill: '#10b981', r: 5 }}
                />
              </LineChart>
            </ResponsiveContainer>
          </CardContent>
        </Card>
      </div>
    </div>
  );
}

# ---- merged ----



# ---- merged ----

import React, { useState } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery } from '@tanstack/react-query';
import { motion } from 'framer-motion';
import { format, startOfDay, isToday } from 'date-fns';
import { Link } from 'react-router-dom';
import { createPageUrl } from '@/utils';
import { Camera, TrendingUp, Utensils, Sparkles, ChevronRight } from 'lucide-react';
import { Button } from "@/components/ui/button";

import DailyProgress from '../components/dashboard/DailyProgress';
import MealHistory from '../components/dashboard/MealHistory';
import WeeklyChart from '../components/analytics/WeeklyChart';

export default function DashboardPage() {
  const [selectedMeal, setSelectedMeal] = useState(null);

  const { data: scans = [], isLoading: scansLoading } = useQuery({
    queryKey: ['foodScans'],
    queryFn: () => base44.entities.FoodScan.list('-created_date', 100),
  });

  const { data: userProfiles = [] } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles[0];
  const todayScans = (scans || []).filter(scan => isToday(new Date(scan.created_date)));

  const getGreeting = () => {
    const hour = new Date().getHours();
    if (hour < 12) return 'Good morning';
    if (hour < 17) return 'Good afternoon';
    return 'Good evening';
  };

  const getMealSuggestion = () => {
    const hour = new Date().getHours();
    if (hour >= 6 && hour < 10) return "Time for breakfast?";
    if (hour >= 11 && hour < 14) return "Ready for lunch?";
    if (hour >= 17 && hour < 21) return "Dinner time?";
    return "Log a snack";
  };

  if (scansLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 pb-24">
      <div className="max-w-2xl mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="mb-8"
        >
          <p className="text-slate-500">{getGreeting()} 👋</p>
          <h1 className="text-2xl font-bold text-slate-800 mt-1">
            {format(new Date(), 'EEEE, MMMM d')}
          </h1>
        </motion.div>

        {/* Quick Scan CTA */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.1 }}
          className="mb-8"
        >
          <Link to={createPageUrl('Scan')}>
            <div className="bg-gradient-to-br from-emerald-500 to-teal-600 rounded-3xl p-6 text-white relative overflow-hidden group cursor-pointer">
              <div className="absolute top-0 right-0 w-32 h-32 bg-white/10 rounded-full -translate-y-1/2 translate-x-1/2" />
              <div className="absolute bottom-0 left-0 w-24 h-24 bg-white/10 rounded-full translate-y-1/2 -translate-x-1/2" />
              
              <div className="relative flex items-center justify-between">
                <div>
                  <p className="text-emerald-100 text-sm">{getMealSuggestion()}</p>
                  <p className="text-xl font-semibold mt-1">Scan your meal</p>
                </div>
                <motion.div 
                  whileHover={{ scale: 1.1 }}
                  className="w-14 h-14 bg-white/20 rounded-2xl flex items-center justify-center"
                >
                  <Camera className="w-7 h-7" />
                </motion.div>
              </div>
            </div>
          </Link>
        </motion.div>

        {/* Daily Progress */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.2 }}
          className="mb-8"
        >
          <DailyProgress todayScans={todayScans} targets={userProfile} />
        </motion.div>

        {/* Weekly Chart */}
        {scans.length > 0 && (
          <motion.div
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.3 }}
            className="mb-8"
          >
            <WeeklyChart scans={scans} targets={userProfile} />
          </motion.div>
        )}

        {/* AI Insights */}
        {todayScans.length >= 2 && (
          <motion.div
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.4 }}
            className="mb-8"
          >
            <div className="bg-gradient-to-br from-violet-50 to-purple-50 rounded-3xl p-6 border border-violet-100">
              <div className="flex items-start gap-4">
                <div className="w-12 h-12 rounded-2xl bg-gradient-to-br from-violet-500 to-purple-600 flex items-center justify-center flex-shrink-0">
                  <Sparkles className="w-6 h-6 text-white" />
                </div>
                <div>
                  <h3 className="font-semibold text-slate-800 mb-1">Daily Insight</h3>
                  <p className="text-sm text-slate-600">
                    {(() => {
                      const totalProtein = todayScans.reduce((sum, s) => sum + (s.protein || 0), 0);
                      const proteinTarget = userProfile?.daily_protein_target || 150;
                      if (totalProtein < proteinTarget * 0.5) {
                        return `You're at ${Math.round(totalProtein)}g protein. Consider adding a protein-rich food to hit your ${proteinTarget}g goal!`;
                      }
                      const avgHealthScore = todayScans.reduce((sum, s) => sum + (s.health_score || 5), 0) / todayScans.length;
                      if (avgHealthScore >= 7) {
                        return "Great choices today! Your meals have been nutritious and balanced.";
                      }
                      return "Try adding more vegetables or whole foods to boost your nutrition score.";
                    })()}
                  </p>
                </div>
              </div>
            </div>
          </motion.div>
        )}

        {/* Meal History */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.5 }}
        >
          <div className="flex items-center justify-between mb-4">
            <h2 className="text-lg font-semibold text-slate-800">Meal History</h2>
            <Link to={createPageUrl('History')}>
              <Button variant="ghost" size="sm" className="text-slate-500">
                View all
                <ChevronRight className="w-4 h-4 ml-1" />
              </Button>
            </Link>
          </div>
          <MealHistory 
            scans={scans.slice(0, 10)} 
            onSelectMeal={setSelectedMeal}
          />
        </motion.div>
      </div>
    </div>
  );
}

# ---- merged ----



# ---- merged ----

import React, { useState } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery } from '@tanstack/react-query';
import { motion, AnimatePresence } from 'framer-motion';
import { format, subDays, startOfDay, endOfDay, isWithinInterval } from 'date-fns';
import { Calendar as CalendarIcon, Filter, X, Flame } from 'lucide-react';
import { Button } from "@/components/ui/button";
import { Calendar } from "@/components/ui/calendar";
import {
  Popover,
  PopoverContent,
  PopoverTrigger,
} from "@/components/ui/popover";
import {
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
} from "@/components/ui/select";
import {
  Sheet,
  SheetContent,
  SheetHeader,
  SheetTitle,
} from "@/components/ui/sheet";

import MealHistory from '../components/dashboard/MealHistory';
import ScanResult from '../components/scanner/ScanResult';

export default function HistoryPage() {
  const [dateRange, setDateRange] = useState({
    from: subDays(new Date(), 7),
    to: new Date(),
  });
  const [mealTypeFilter, setMealTypeFilter] = useState('all');
  const [selectedMeal, setSelectedMeal] = useState(null);

  const { data: scans = [], isLoading } = useQuery({
    queryKey: ['foodScans'],
    queryFn: () => base44.entities.FoodScan.list('-created_date', 500),
  });

  const { data: userProfiles = [] } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles[0];

  const filteredScans = (scans || []).filter(scan => {
    const scanDate = new Date(scan.created_date);
    const inDateRange = isWithinInterval(scanDate, {
      start: startOfDay(dateRange.from),
      end: endOfDay(dateRange.to),
    });
    const matchesMealType = mealTypeFilter === 'all' || scan.meal_type === mealTypeFilter;
    return inDateRange && matchesMealType;
  });

  const stats = (filteredScans || []).reduce((acc, scan) => ({
    totalCalories: acc.totalCalories + (scan.calories || 0),
    totalProtein: acc.totalProtein + (scan.protein || 0),
    totalCarbs: acc.totalCarbs + (scan.carbs || 0),
    totalFats: acc.totalFats + (scan.fats || 0),
    avgHealthScore: acc.avgHealthScore + (scan.health_score || 0),
    count: acc.count + 1,
  }), { totalCalories: 0, totalProtein: 0, totalCarbs: 0, totalFats: 0, avgHealthScore: 0, count: 0 });

  if (stats.count > 0) {
    stats.avgHealthScore = Math.round(stats.avgHealthScore / stats.count * 10) / 10;
  }

  if (isLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 pb-24">
      <div className="max-w-2xl mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="mb-6"
        >
          <h1 className="text-2xl font-bold text-slate-800">Meal History</h1>
          <p className="text-slate-500 text-sm mt-1">Review your nutrition journey</p>
        </motion.div>

        {/* Filters */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.1 }}
          className="flex flex-wrap gap-3 mb-6"
        >
          <Popover>
            <PopoverTrigger asChild>
              <Button variant="outline" className="rounded-2xl h-11">
                <CalendarIcon className="w-4 h-4 mr-2" />
                {format(dateRange.from, 'MMM d')} - {format(dateRange.to, 'MMM d')}
              </Button>
            </PopoverTrigger>
            <PopoverContent className="w-auto p-0" align="start">
              <Calendar
                mode="range"
                selected={dateRange}
                onSelect={(range) => range && setDateRange(range)}
                numberOfMonths={1}
              />
            </PopoverContent>
          </Popover>

          <Select value={mealTypeFilter} onValueChange={setMealTypeFilter}>
            <SelectTrigger className="w-40 rounded-2xl h-11">
              <Filter className="w-4 h-4 mr-2" />
              <SelectValue placeholder="Meal type" />
            </SelectTrigger>
            <SelectContent>
              <SelectItem value="all">All Meals</SelectItem>
              <SelectItem value="breakfast">🌅 Breakfast</SelectItem>
              <SelectItem value="lunch">☀️ Lunch</SelectItem>
              <SelectItem value="dinner">🌙 Dinner</SelectItem>
              <SelectItem value="snack">🍎 Snack</SelectItem>
            </SelectContent>
          </Select>
        </motion.div>

        {/* Period Stats */}
        {filteredScans.length > 0 && (
          <motion.div
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.2 }}
            className="bg-white rounded-3xl p-6 border border-slate-100 shadow-sm mb-6"
          >
            <h3 className="text-sm font-medium text-slate-500 mb-4">Period Summary</h3>
            <div className="grid grid-cols-2 md:grid-cols-4 gap-4">
              <div className="text-center p-3 bg-orange-50 rounded-2xl">
                <Flame className="w-5 h-5 text-orange-500 mx-auto mb-1" />
                <p className="text-xl font-bold text-slate-800">{Math.round(stats.totalCalories).toLocaleString()}</p>
                <p className="text-xs text-slate-500">Total kcal</p>
              </div>
              <div className="text-center p-3 bg-rose-50 rounded-2xl">
                <p className="text-xl font-bold text-slate-800">{Math.round(stats.totalProtein)}g</p>
                <p className="text-xs text-slate-500">Protein</p>
              </div>
              <div className="text-center p-3 bg-amber-50 rounded-2xl">
                <p className="text-xl font-bold text-slate-800">{Math.round(stats.totalCarbs)}g</p>
                <p className="text-xs text-slate-500">Carbs</p>
              </div>
              <div className="text-center p-3 bg-emerald-50 rounded-2xl">
                <p className="text-xl font-bold text-slate-800">{stats.avgHealthScore}</p>
                <p className="text-xs text-slate-500">Avg Score</p>
              </div>
            </div>
          </motion.div>
        )}

        {/* Meal List */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.3 }}
        >
          <MealHistory 
            scans={filteredScans} 
            onSelectMeal={setSelectedMeal}
          />
        </motion.div>

        {/* Meal Detail Sheet */}
        <Sheet open={!!selectedMeal} onOpenChange={() => setSelectedMeal(null)}>
          <SheetContent side="bottom" className="h-[90vh] rounded-t-3xl">
            <SheetHeader className="mb-4">
              <div className="flex items-center justify-between">
                <SheetTitle>Meal Details</SheetTitle>
                <Button
                  variant="ghost"
                  size="icon"
                  onClick={() => setSelectedMeal(null)}
                  className="rounded-full"
                >
                  <X className="w-5 h-5" />
                </Button>
              </div>
            </SheetHeader>
            {selectedMeal && (
              <div className="overflow-y-auto max-h-[calc(90vh-80px)]">
                <ScanResult
                  result={selectedMeal}
                  onAddToLog={() => {}}
                  onScanAgain={() => setSelectedMeal(null)}
                  userAllergies={userProfile?.allergies || []}
                />
              </div>
            )}
          </SheetContent>
        </Sheet>
      </div>
    </div>
  );
}

# ---- merged ----

import React, { useState } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery, useMutation, useQueryClient } from '@tanstack/react-query';
import { motion } from 'framer-motion';
import { toast } from 'sonner';
import { 
  User, Target, Scale, Ruler, Activity, AlertCircle, 
  Flame, Beef, Wheat, Droplets, ChevronRight, Check, Pencil
} from 'lucide-react';
import { Button } from "@/components/ui/button";
import { Input } from "@/components/ui/input";
import { Badge } from "@/components/ui/badge";
import {
  Sheet,
  SheetContent,
  SheetHeader,
  SheetTitle,
  SheetFooter,
} from "@/components/ui/sheet";
import {
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
} from "@/components/ui/select";

const dietaryGoals = {
  weight_loss: { label: 'Lose Weight', emoji: '🔥' },
  muscle_gain: { label: 'Build Muscle', emoji: '💪' },
  maintenance: { label: 'Maintain', emoji: '⚖️' },
  health_improvement: { label: 'Eat Healthier', emoji: '🥗' },
};

const dietTypes = {
  none: { label: 'No Preference', emoji: '🍽️' },
  vegan: { label: 'Vegan', emoji: '🌱' },
  vegetarian: { label: 'Vegetarian', emoji: '🥬' },
  keto: { label: 'Keto', emoji: '🥑' },
  paleo: { label: 'Paleo', emoji: '🥩' },
  mediterranean: { label: 'Mediterranean', emoji: '🫒' },
  low_carb: { label: 'Low Carb', emoji: '🥚' },
  diabetic_friendly: { label: 'Diabetic Friendly', emoji: '💚' },
};

const activityLevels = {
  sedentary: { label: 'Sedentary', desc: 'Little to no exercise' },
  light: { label: 'Light', desc: '1-3 days/week' },
  moderate: { label: 'Moderate', desc: '3-5 days/week' },
  active: { label: 'Active', desc: '6-7 days/week' },
  very_active: { label: 'Very Active', desc: 'Intense daily exercise' },
};

const commonAllergens = ['Peanuts', 'Tree Nuts', 'Milk', 'Eggs', 'Wheat', 'Soy', 'Fish', 'Shellfish', 'Sesame'];

export default function ProfilePage() {
  const [editSheet, setEditSheet] = useState(null);
  const [editData, setEditData] = useState({});
  
  const queryClient = useQueryClient();

  const { data: userProfiles = [], isLoading } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles[0];

  const updateProfileMutation = useMutation({
    mutationFn: (data) => base44.entities.UserProfile.update(userProfile.id, data),
    onSuccess: () => {
      queryClient.invalidateQueries({ queryKey: ['userProfile'] });
      toast.success('Profile updated!');
      setEditSheet(null);
    },
  });

  const openEdit = (section, data) => {
    setEditData(data);
    setEditSheet(section);
  };

  const handleSave = () => {
    updateProfileMutation.mutate(editData);
  };

  const toggleAllergen = (allergen) => {
    const current = editData.allergies || [];
    setEditData(prev => ({
      ...prev,
      allergies: current.includes(allergen)
        ? current.filter(a => a !== allergen)
        : [...current, allergen]
    }));
  };

  if (isLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  if (!userProfile) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center p-4">
        <div className="text-center">
          <User className="w-16 h-16 text-slate-300 mx-auto mb-4" />
          <h2 className="text-xl font-semibold text-slate-800 mb-2">No Profile Yet</h2>
          <p className="text-slate-500">Start by scanning your first meal!</p>
        </div>
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 pb-24">
      <div className="max-w-lg mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="mb-8"
        >
          <h1 className="text-2xl font-bold text-slate-800">Profile</h1>
          <p className="text-slate-500 text-sm mt-1">Manage your nutrition settings</p>
        </motion.div>

        {/* Goals Section */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.1 }}
          className="mb-6"
        >
          <div 
            onClick={() => openEdit('goals', { 
              dietary_goal: userProfile.dietary_goal,
              diet_type: userProfile.diet_type 
            })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="flex items-center justify-between">
              <div className="flex items-center gap-4">
                <div className="w-12 h-12 rounded-2xl bg-gradient-to-br from-emerald-500 to-teal-600 flex items-center justify-center">
                  <Target className="w-6 h-6 text-white" />
                </div>
                <div>
                  <p className="font-semibold text-slate-800">
                    {dietaryGoals[userProfile.dietary_goal]?.emoji} {dietaryGoals[userProfile.dietary_goal]?.label || 'Set Goal'}
                  </p>
                  <p className="text-sm text-slate-500">
                    {dietTypes[userProfile.diet_type]?.emoji} {dietTypes[userProfile.diet_type]?.label || 'No diet preference'}
                  </p>
                </div>
              </div>
              <ChevronRight className="w-5 h-5 text-slate-400" />
            </div>
          </div>
        </motion.div>

        {/* Daily Targets */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.2 }}
          className="mb-6"
        >
          <h3 className="text-sm font-medium text-slate-500 mb-3 px-1">Daily Targets</h3>
          <div 
            onClick={() => openEdit('targets', {
              daily_calorie_target: userProfile.daily_calorie_target,
              daily_protein_target: userProfile.daily_protein_target,
              daily_carbs_target: userProfile.daily_carbs_target,
              daily_fats_target: userProfile.daily_fats_target,
            })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="grid grid-cols-2 gap-4">
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-orange-50 flex items-center justify-center">
                  <Flame className="w-5 h-5 text-orange-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_calorie_target || 2000}</p>
                  <p className="text-xs text-slate-500">Calories</p>
                </div>
              </div>
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-rose-50 flex items-center justify-center">
                  <Beef className="w-5 h-5 text-rose-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_protein_target || 150}g</p>
                  <p className="text-xs text-slate-500">Protein</p>
                </div>
              </div>
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-amber-50 flex items-center justify-center">
                  <Wheat className="w-5 h-5 text-amber-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_carbs_target || 250}g</p>
                  <p className="text-xs text-slate-500">Carbs</p>
                </div>
              </div>
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-blue-50 flex items-center justify-center">
                  <Droplets className="w-5 h-5 text-blue-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_fats_target || 65}g</p>
                  <p className="text-xs text-slate-500">Fats</p>
                </div>
              </div>
            </div>
            <div className="flex justify-end mt-3">
              <Pencil className="w-4 h-4 text-slate-400" />
            </div>
          </div>
        </motion.div>

        {/* Body Stats */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.3 }}
          className="mb-6"
        >
          <h3 className="text-sm font-medium text-slate-500 mb-3 px-1">Body Stats</h3>
          <div 
            onClick={() => openEdit('stats', {
              weight: userProfile.weight,
              height: userProfile.height,
              age: userProfile.age,
              activity_level: userProfile.activity_level,
            })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="grid grid-cols-3 gap-4 text-center">
              <div>
                <Scale className="w-5 h-5 text-slate-400 mx-auto mb-1" />
                <p className="font-semibold text-slate-800">{userProfile.weight || '--'} kg</p>
                <p className="text-xs text-slate-500">Weight</p>
              </div>
              <div>
                <Ruler className="w-5 h-5 text-slate-400 mx-auto mb-1" />
                <p className="font-semibold text-slate-800">{userProfile.height || '--'} cm</p>
                <p className="text-xs text-slate-500">Height</p>
              </div>
              <div>
                <Activity className="w-5 h-5 text-slate-400 mx-auto mb-1" />
                <p className="font-semibold text-slate-800 capitalize">{activityLevels[userProfile.activity_level]?.label || '--'}</p>
                <p className="text-xs text-slate-500">Activity</p>
              </div>
            </div>
          </div>
        </motion.div>

        {/* Allergies */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.4 }}
        >
          <h3 className="text-sm font-medium text-slate-500 mb-3 px-1">Allergies & Alerts</h3>
          <div 
            onClick={() => openEdit('allergies', { allergies: userProfile.allergies || [] })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="flex items-start gap-3">
              <AlertCircle className="w-5 h-5 text-red-500 mt-0.5" />
              <div className="flex-1">
                {userProfile.allergies?.length > 0 ? (
                  <div className="flex flex-wrap gap-2">
                    {userProfile.allergies.map(allergen => (
                      <Badge key={allergen} variant="secondary" className="bg-red-50 text-red-700 border-red-200">
                        {allergen}
                      </Badge>
                    ))}
                  </div>
                ) : (
                  <p className="text-slate-500">No allergies set</p>
                )}
              </div>
              <ChevronRight className="w-5 h-5 text-slate-400" />
            </div>
          </div>
        </motion.div>

        {/* Edit Sheets */}
        <Sheet open={editSheet === 'goals'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Goals</SheetTitle>
            </SheetHeader>
            <div className="py-6 space-y-4">
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Dietary Goal</label>
                <Select value={editData.dietary_goal} onValueChange={(v) => setEditData(p => ({ ...p, dietary_goal: v }))}>
                  <SelectTrigger className="h-12 rounded-xl">
                    <SelectValue placeholder="Select goal" />
                  </SelectTrigger>
                  <SelectContent>
                    {Object.entries(dietaryGoals).map(([key, { label, emoji }]) => (
                      <SelectItem key={key} value={key}>{emoji} {label}</SelectItem>
                    ))}
                  </SelectContent>
                </Select>
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Diet Type</label>
                <Select value={editData.diet_type} onValueChange={(v) => setEditData(p => ({ ...p, diet_type: v }))}>
                  <SelectTrigger className="h-12 rounded-xl">
                    <SelectValue placeholder="Select diet" />
                  </SelectTrigger>
                  <SelectContent>
                    {Object.entries(dietTypes).map(([key, { label, emoji }]) => (
                      <SelectItem key={key} value={key}>{emoji} {label}</SelectItem>
                    ))}
                  </SelectContent>
                </Select>
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>

        <Sheet open={editSheet === 'targets'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Daily Targets</SheetTitle>
            </SheetHeader>
            <div className="py-6 grid grid-cols-2 gap-4">
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Calories</label>
                <Input 
                  type="number" 
                  value={editData.daily_calorie_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_calorie_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Protein (g)</label>
                <Input 
                  type="number" 
                  value={editData.daily_protein_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_protein_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Carbs (g)</label>
                <Input 
                  type="number" 
                  value={editData.daily_carbs_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_carbs_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Fats (g)</label>
                <Input 
                  type="number" 
                  value={editData.daily_fats_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_fats_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>

        <Sheet open={editSheet === 'stats'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Body Stats</SheetTitle>
            </SheetHeader>
            <div className="py-6 space-y-4">
              <div className="grid grid-cols-3 gap-4">
                <div>
                  <label className="text-sm font-medium text-slate-600 mb-2 block">Weight (kg)</label>
                  <Input 
                    type="number" 
                    value={editData.weight || ''} 
                    onChange={(e) => setEditData(p => ({ ...p, weight: parseFloat(e.target.value) }))}
                    className="h-12 rounded-xl"
                  />
                </div>
                <div>
                  <label className="text-sm font-medium text-slate-600 mb-2 block">Height (cm)</label>
                  <Input 
                    type="number" 
                    value={editData.height || ''} 
                    onChange={(e) => setEditData(p => ({ ...p, height: parseFloat(e.target.value) }))}
                    className="h-12 rounded-xl"
                  />
                </div>
                <div>
                  <label className="text-sm font-medium text-slate-600 mb-2 block">Age</label>
                  <Input 
                    type="number" 
                    value={editData.age || ''} 
                    onChange={(e) => setEditData(p => ({ ...p, age: parseInt(e.target.value) }))}
                    className="h-12 rounded-xl"
                  />
                </div>
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Activity Level</label>
                <Select value={editData.activity_level} onValueChange={(v) => setEditData(p => ({ ...p, activity_level: v }))}>
                  <SelectTrigger className="h-12 rounded-xl">
                    <SelectValue placeholder="Select activity" />
                  </SelectTrigger>
                  <SelectContent>
                    {Object.entries(activityLevels).map(([key, { label, desc }]) => (
                      <SelectItem key={key} value={key}>{label} - {desc}</SelectItem>
                    ))}
                  </SelectContent>
                </Select>
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>

        <Sheet open={editSheet === 'allergies'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Allergies</SheetTitle>
            </SheetHeader>
            <div className="py-6">
              <p className="text-sm text-slate-500 mb-4">Tap to toggle allergens you want to be alerted about</p>
              <div className="flex flex-wrap gap-2">
                {commonAllergens.map(allergen => (
                  <Badge
                    key={allergen}
                    variant={editData.allergies?.includes(allergen) ? 'default' : 'outline'}
                    className={`px-4 py-2 text-sm cursor-pointer transition-all ${
                      editData.allergies?.includes(allergen)
                        ? 'bg-red-500 hover:bg-red-600 border-red-500'
                        : 'hover:border-slate-400'
                    }`}
                    onClick={() => toggleAllergen(allergen)}
                  >
                    {editData.allergies?.includes(allergen) && (
                      <Check className="w-3 h-3 mr-1" />
                    )}
                    {allergen}
                  </Badge>
                ))}
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>
      </div>
    </div>
  );
}

# ---- merged ----

{
  "name": "UserProfile",
  "type": "object",
  "properties": {
    "dietary_goal": {
      "type": "string",
      "enum": [
        "weight_loss",
        "muscle_gain",
        "maintenance",
        "health_improvement"
      ],
      "description": "Primary dietary goal"
    },
    "diet_type": {
      "type": "string",
      "enum": [
        "none",
        "vegan",
        "vegetarian",
        "keto",
        "paleo",
        "mediterranean",
        "low_carb",
        "diabetic_friendly"
      ],
      "description": "Diet preference"
    },
    "allergies": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Food allergies"
    },
    "daily_calorie_target": {
      "type": "number",
      "description": "Daily calorie goal"
    },
    "daily_protein_target": {
      "type": "number",
      "description": "Daily protein goal in grams"
    },
    "daily_carbs_target": {
      "type": "number",
      "description": "Daily carbs goal in grams"
    },
    "daily_fats_target": {
      "type": "number",
      "description": "Daily fats goal in grams"
    },
    "weight": {
      "type": "number",
      "description": "Current weight in kg"
    },
    "height": {
      "type": "number",
      "description": "Height in cm"
    },
    "age": {
      "type": "number",
      "description": "Age in years"
    },
    "activity_level": {
      "type": "string",
      "enum": [
        "sedentary",
        "light",
        "moderate",
        "active",
        "very_active"
      ],
      "description": "Activity level"
    },
    "onboarding_complete": {
      "type": "boolean",
      "description": "Whether user has completed onboarding"
    }
  },
  "required": []
}

# ---- merged ----

import React, { useState } from 'react';
import { motion, AnimatePresence } from 'framer-motion';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Input } from "@/components/ui/input";
import { Label } from "@/components/ui/label";
import { Badge } from "@/components/ui/badge";
import { ChevronRight, ChevronLeft, Target, Utensils, AlertCircle, Activity, User } from 'lucide-react';
import { cn } from "@/lib/utils";

export default function ProfileSetup({ onComplete }) {
  const [step, setStep] = useState(1);
  const [profile, setProfile] = useState({
    dietary_goal: '',
    diet_type: 'none',
    allergies: [],
    weight: '',
    height: '',
    age: '',
    activity_level: '',
    daily_calorie_target: 2000,
    daily_protein_target: 150,
    daily_carbs_target: 250,
    daily_fats_target: 65
  });

  const goals = [
    { value: 'weight_loss', label: 'Weight Loss', icon: '🎯', desc: 'Lose weight healthily' },
    { value: 'muscle_gain', label: 'Muscle Gain', icon: '💪', desc: 'Build lean muscle' },
    { value: 'maintenance', label: 'Maintenance', icon: '⚖️', desc: 'Maintain current weight' },
    { value: 'health_improvement', label: 'Health', icon: '❤️', desc: 'Improve overall health' }
  ];

  const diets = [
    { value: 'none', label: 'No Preference' },
    { value: 'vegan', label: 'Vegan' },
    { value: 'vegetarian', label: 'Vegetarian' },
    { value: 'keto', label: 'Keto' },
    { value: 'paleo', label: 'Paleo' },
    { value: 'mediterranean', label: 'Mediterranean' },
    { value: 'low_carb', label: 'Low Carb' },
    { value: 'diabetic_friendly', label: 'Diabetic Friendly' }
  ];

  const commonAllergies = ['Peanuts', 'Tree Nuts', 'Dairy', 'Eggs', 'Soy', 'Wheat', 'Fish', 'Shellfish'];
  
  const activityLevels = [
    { value: 'sedentary', label: 'Sedentary', desc: 'Little or no exercise' },
    { value: 'light', label: 'Light', desc: '1-3 days/week' },
    { value: 'moderate', label: 'Moderate', desc: '3-5 days/week' },
    { value: 'active', label: 'Active', desc: '6-7 days/week' },
    { value: 'very_active', label: 'Very Active', desc: 'Intense daily exercise' }
  ];

  const toggleAllergy = (allergy) => {
    setProfile(prev => ({
      ...prev,
      allergies: prev.allergies.includes(allergy)
        ? prev.allergies.filter(a => a !== allergy)
        : [...prev.allergies, allergy]
    }));
  };

  const calculateTargets = () => {
    const weight = parseFloat(profile.weight);
    const height = parseFloat(profile.height);
    const age = parseFloat(profile.age);
    
    if (!weight || !height || !age) return;

    // Basic BMR calculation (Mifflin-St Jeor)
    let bmr = 10 * weight + 6.25 * height - 5 * age + 5;
    
    const activityMultipliers = {
      sedentary: 1.2, light: 1.375, moderate: 1.55, active: 1.725, very_active: 1.9
    };
    
    let tdee = bmr * (activityMultipliers[profile.activity_level] || 1.5);
    
    if (profile.dietary_goal === 'weight_loss') tdee -= 500;
    if (profile.dietary_goal === 'muscle_gain') tdee += 300;
    
    setProfile(prev => ({
      ...prev,
      daily_calorie_target: Math.round(tdee),
      daily_protein_target: Math.round(weight * 2),
      daily_carbs_target: Math.round((tdee * 0.45) / 4),
      daily_fats_target: Math.round((tdee * 0.3) / 9)
    }));
  };

  const handleNext = () => {
    if (step === 4) {
      calculateTargets();
      setTimeout(() => onComplete({ ...profile, onboarding_complete: true }), 100);
    } else {
      setStep(step + 1);
    }
  };

  const canProceed = () => {
    if (step === 1) return profile.dietary_goal;
    if (step === 2) return true;
    if (step === 3) return true;
    if (step === 4) return profile.weight && profile.height && profile.age && profile.activity_level;
    return false;
  };

  return (
    <div className="min-h-screen bg-gradient-to-br from-emerald-50 via-teal-50 to-cyan-50 p-6 flex items-center justify-center">
      <div className="w-full max-w-2xl">
        {/* Progress Bar */}
        <div className="mb-8">
          <div className="flex justify-between mb-2">
            <span className="text-sm font-medium text-slate-700">Step {step} of 4</span>
            <span className="text-sm text-slate-500">{Math.round((step / 4) * 100)}%</span>
          </div>
          <div className="h-2 bg-slate-200 rounded-full overflow-hidden">
            <motion.div 
              className="h-full bg-gradient-to-r from-emerald-500 to-teal-600"
              initial={{ width: 0 }}
              animate={{ width: `${(step / 4) * 100}%` }}
              transition={{ duration: 0.3 }}
            />
          </div>
        </div>

        <AnimatePresence mode="wait">
          {/* Step 1: Goal */}
          {step === 1 && (
            <motion.div
              key="step1"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-emerald-100 rounded-xl">
                      <Target className="w-6 h-6 text-emerald-600" />
                    </div>
                    <CardTitle className="text-2xl">What's your goal?</CardTitle>
                  </div>
                  <p className="text-slate-600">Help us personalize your experience</p>
                </CardHeader>
                <CardContent className="grid grid-cols-1 md:grid-cols-2 gap-4">
                  {goals.map((goal) => (
                    <motion.button
                      key={goal.value}
                      whileHover={{ scale: 1.02 }}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile({ ...profile, dietary_goal: goal.value })}
                      className={cn(
                        "p-6 rounded-2xl border-2 text-left transition-all",
                        profile.dietary_goal === goal.value
                          ? "border-emerald-500 bg-emerald-50"
                          : "border-slate-200 hover:border-slate-300"
                      )}
                    >
                      <div className="text-4xl mb-3">{goal.icon}</div>
                      <h3 className="font-semibold text-lg mb-1">{goal.label}</h3>
                      <p className="text-sm text-slate-600">{goal.desc}</p>
                    </motion.button>
                  ))}
                </CardContent>
              </Card>
            </motion.div>
          )}

          {/* Step 2: Diet Type */}
          {step === 2 && (
            <motion.div
              key="step2"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-teal-100 rounded-xl">
                      <Utensils className="w-6 h-6 text-teal-600" />
                    </div>
                    <CardTitle className="text-2xl">Dietary preference?</CardTitle>
                  </div>
                  <p className="text-slate-600">Optional - helps with food recommendations</p>
                </CardHeader>
                <CardContent className="grid grid-cols-2 md:grid-cols-3 gap-3">
                  {diets.map((diet) => (
                    <Button
                      key={diet.value}
                      variant={profile.diet_type === diet.value ? "default" : "outline"}
                      onClick={() => setProfile({ ...profile, diet_type: diet.value })}
                      className={cn(
                        "h-auto py-4",
                        profile.diet_type === diet.value && "bg-teal-600 hover:bg-teal-700"
                      )}
                    >
                      {diet.label}
                    </Button>
                  ))}
                </CardContent>
              </Card>
            </motion.div>
          )}

          {/* Step 3: Allergies */}
          {step === 3 && (
            <motion.div
              key="step3"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-red-100 rounded-xl">
                      <AlertCircle className="w-6 h-6 text-red-600" />
                    </div>
                    <CardTitle className="text-2xl">Any allergies?</CardTitle>
                  </div>
                  <p className="text-slate-600">We'll alert you when scanning food</p>
                </CardHeader>
                <CardContent className="space-y-4">
                  <div className="flex flex-wrap gap-2">
                    {commonAllergies.map((allergy) => (
                      <Badge
                        key={allergy}
                        variant={profile.allergies.includes(allergy) ? "default" : "outline"}
                        className={cn(
                          "cursor-pointer py-2 px-4 text-sm",
                          profile.allergies.includes(allergy) && "bg-red-600 hover:bg-red-700"
                        )}
                        onClick={() => toggleAllergy(allergy)}
                      >
                        {allergy}
                      </Badge>
                    ))}
                  </div>
                  <p className="text-xs text-slate-500">Tap to select/deselect</p>
                </CardContent>
              </Card>
            </motion.div>
          )}

          {/* Step 4: Stats */}
          {step === 4 && (
            <motion.div
              key="step4"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-purple-100 rounded-xl">
                      <User className="w-6 h-6 text-purple-600" />
                    </div>
                    <CardTitle className="text-2xl">About you</CardTitle>
                  </div>
                  <p className="text-slate-600">To calculate your daily targets</p>
                </CardHeader>
                <CardContent className="space-y-6">
                  <div className="grid grid-cols-3 gap-4">
                    <div>
                      <Label htmlFor="weight">Weight (kg)</Label>
                      <Input
                        id="weight"
                        type="number"
                        value={profile.weight}
                        onChange={(e) => setProfile({ ...profile, weight: e.target.value })}
                        placeholder="70"
                        className="mt-2"
                      />
                    </div>
                    <div>
                      <Label htmlFor="height">Height (cm)</Label>
                      <Input
                        id="height"
                        type="number"
                        value={profile.height}
                        onChange={(e) => setProfile({ ...profile, height: e.target.value })}
                        placeholder="170"
                        className="mt-2"
                      />
                    </div>
                    <div>
                      <Label htmlFor="age">Age</Label>
                      <Input
                        id="age"
                        type="number"
                        value={profile.age}
                        onChange={(e) => setProfile({ ...profile, age: e.target.value })}
                        placeholder="30"
                        className="mt-2"
                      />
                    </div>
                  </div>

                  <div>
                    <Label className="mb-3 block">Activity Level</Label>
                    <div className="space-y-2">
                      {activityLevels.map((level) => (
                        <button
                          key={level.value}
                          onClick={() => setProfile({ ...profile, activity_level: level.value })}
                          className={cn(
                            "w-full p-4 rounded-xl border-2 text-left transition-all",
                            profile.activity_level === level.value
                              ? "border-purple-500 bg-purple-50"
                              : "border-slate-200 hover:border-slate-300"
                          )}
                        >
                          <div className="flex justify-between items-center">
                            <div>
                              <p className="font-semibold">{level.label}</p>
                              <p className="text-sm text-slate-600">{level.desc}</p>
                            </div>
                            {profile.activity_level === level.value && (
                              <Activity className="w-5 h-5 text-purple-600" />
                            )}
                          </div>
                        </button>
                      ))}
                    </div>
                  </div>
                </CardContent>
              </Card>
            </motion.div>
          )}
        </AnimatePresence>

        {/* Navigation */}
        <div className="flex gap-3 mt-6">
          {step > 1 && (
            <Button
              variant="outline"
              onClick={() => setStep(step - 1)}
              className="px-6"
            >
              <ChevronLeft className="w-4 h-4 mr-1" />
              Back
            </Button>
          )}
          <Button
            onClick={handleNext}
            disabled={!canProceed()}
            className="flex-1 bg-gradient-to-r from-emerald-600 to-teal-600 hover:from-emerald-700 hover:to-teal-700"
          >
            {step === 4 ? 'Complete Setup' : 'Continue'}
            <ChevronRight className="w-4 h-4 ml-1" />
          </Button>
        </div>
      </div>
    </div>
  );
}

# ---- merged ----

import React, { useState } from 'react';
import { motion, AnimatePresence } from 'framer-motion';
import { Button } from "@/components/ui/button";
import { Input } from "@/components/ui/input";
import { Badge } from "@/components/ui/badge";
import { 
  ChevronRight, ChevronLeft, Target, Scale, Ruler, 
  Activity, AlertCircle, Sparkles, Check
} from 'lucide-react';

const dietaryGoals = [
  { id: 'weight_loss', label: 'Lose Weight', emoji: '🔥', desc: 'Create a calorie deficit' },
  { id: 'muscle_gain', label: 'Build Muscle', emoji: '💪', desc: 'High protein focus' },
  { id: 'maintenance', label: 'Maintain', emoji: '⚖️', desc: 'Stay balanced' },
  { id: 'health_improvement', label: 'Eat Healthier', emoji: '🥗', desc: 'Improve nutrition' },
];

const dietTypes = [
  { id: 'none', label: 'No Preference', emoji: '🍽️' },
  { id: 'vegan', label: 'Vegan', emoji: '🌱' },
  { id: 'vegetarian', label: 'Vegetarian', emoji: '🥬' },
  { id: 'keto', label: 'Keto', emoji: '🥑' },
  { id: 'paleo', label: 'Paleo', emoji: '🥩' },
  { id: 'mediterranean', label: 'Mediterranean', emoji: '🫒' },
  { id: 'low_carb', label: 'Low Carb', emoji: '🥚' },
  { id: 'diabetic_friendly', label: 'Diabetic Friendly', emoji: '💚' },
];

const activityLevels = [
  { id: 'sedentary', label: 'Sedentary', desc: 'Little to no exercise' },
  { id: 'light', label: 'Light', desc: '1-3 days/week' },
  { id: 'moderate', label: 'Moderate', desc: '3-5 days/week' },
  { id: 'active', label: 'Active', desc: '6-7 days/week' },
  { id: 'very_active', label: 'Very Active', desc: 'Intense daily exercise' },
];

const commonAllergens = ['Peanuts', 'Tree Nuts', 'Milk', 'Eggs', 'Wheat', 'Soy', 'Fish', 'Shellfish', 'Sesame'];

const SlideTransition = ({ children }) => (
  <motion.div
    initial={{ opacity: 0, x: 50 }}
    animate={{ opacity: 1, x: 0 }}
    exit={{ opacity: 0, x: -50 }}
    transition={{ duration: 0.3 }}
  >
    {children}
  </motion.div>
);

export default function OnboardingFlow({ onComplete }) {
  const [step, setStep] = useState(0);
  const [profile, setProfile] = useState({
    dietary_goal: '',
    diet_type: 'none',
    allergies: [],
    weight: '',
    height: '',
    age: '',
    activity_level: 'moderate',
    daily_calorie_target: 2000,
    daily_protein_target: 150,
    daily_carbs_target: 250,
    daily_fats_target: 65,
  });

  const totalSteps = 5;

  const calculateTargets = () => {
    const weight = parseFloat(profile.weight) || 70;
    const height = parseFloat(profile.height) || 170;
    const age = parseFloat(profile.age) || 30;
    
    // Basic BMR calculation (Mifflin-St Jeor)
    let bmr = 10 * weight + 6.25 * height - 5 * age + 5;
    
    const activityMultipliers = {
      sedentary: 1.2,
      light: 1.375,
      moderate: 1.55,
      active: 1.725,
      very_active: 1.9
    };
    
    let tdee = bmr * activityMultipliers[profile.activity_level];
    
    // Adjust based on goal
    if (profile.dietary_goal === 'weight_loss') {
      tdee -= 500;
    } else if (profile.dietary_goal === 'muscle_gain') {
      tdee += 300;
    }
    
    const calories = Math.round(tdee);
    const protein = Math.round(weight * (profile.dietary_goal === 'muscle_gain' ? 2.2 : 1.6));
    const fats = Math.round((calories * 0.25) / 9);
    const carbs = Math.round((calories - (protein * 4) - (fats * 9)) / 4);
    
    return {
      daily_calorie_target: calories,
      daily_protein_target: protein,
      daily_carbs_target: carbs,
      daily_fats_target: fats,
    };
  };

  const handleComplete = () => {
    const targets = calculateTargets();
    onComplete({ 
      ...profile, 
      ...targets,
      onboarding_complete: true 
    });
  };

  const toggleAllergen = (allergen) => {
    setProfile(prev => ({
      ...prev,
      allergies: prev.allergies.includes(allergen)
        ? prev.allergies.filter(a => a !== allergen)
        : [...prev.allergies, allergen]
    }));
  };

  const canProceed = () => {
    switch (step) {
      case 0: return profile.dietary_goal;
      case 1: return true;
      case 2: return profile.weight && profile.height && profile.age;
      case 3: return profile.activity_level;
      case 4: return true;
      default: return true;
    }
  };

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex flex-col">
      {/* Progress */}
      <div className="p-6">
        <div className="flex gap-2">
          {Array.from({ length: totalSteps }, (_, i) => (
            <div 
              key={i}
              className={`h-1.5 flex-1 rounded-full transition-colors ${
                i <= step ? 'bg-emerald-500' : 'bg-slate-200'
              }`}
            />
          ))}
        </div>
      </div>

      {/* Content */}
      <div className="flex-1 px-6 pb-6">
        <AnimatePresence mode="wait">
          {step === 0 && (
            <SlideTransition key="goal">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">What's your goal?</h1>
                  <p className="text-slate-500 mt-1">We'll personalize your experience</p>
                </div>
                <div className="grid gap-3">
                  {dietaryGoals.map(goal => (
                    <motion.button
                      key={goal.id}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile(p => ({ ...p, dietary_goal: goal.id }))}
                      className={`p-4 rounded-2xl border-2 text-left transition-all ${
                        profile.dietary_goal === goal.id
                          ? 'border-emerald-500 bg-emerald-50'
                          : 'border-slate-200 bg-white hover:border-slate-300'
                      }`}
                    >
                      <div className="flex items-center gap-4">
                        <span className="text-3xl">{goal.emoji}</span>
                        <div className="flex-1">
                          <p className="font-semibold text-slate-800">{goal.label}</p>
                          <p className="text-sm text-slate-500">{goal.desc}</p>
                        </div>
                        {profile.dietary_goal === goal.id && (
                          <Check className="w-5 h-5 text-emerald-500" />
                        )}
                      </div>
                    </motion.button>
                  ))}
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 1 && (
            <SlideTransition key="diet">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Dietary preference?</h1>
                  <p className="text-slate-500 mt-1">Select your eating style</p>
                </div>
                <div className="grid grid-cols-2 gap-3">
                  {dietTypes.map(diet => (
                    <motion.button
                      key={diet.id}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile(p => ({ ...p, diet_type: diet.id }))}
                      className={`p-4 rounded-2xl border-2 text-center transition-all ${
                        profile.diet_type === diet.id
                          ? 'border-emerald-500 bg-emerald-50'
                          : 'border-slate-200 bg-white hover:border-slate-300'
                      }`}
                    >
                      <span className="text-2xl block mb-1">{diet.emoji}</span>
                      <p className="font-medium text-slate-800 text-sm">{diet.label}</p>
                    </motion.button>
                  ))}
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 2 && (
            <SlideTransition key="stats">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Your stats</h1>
                  <p className="text-slate-500 mt-1">Help us calculate your targets</p>
                </div>
                <div className="space-y-4">
                  <div className="bg-white rounded-2xl p-4 border border-slate-200">
                    <div className="flex items-center gap-3 mb-2">
                      <Scale className="w-5 h-5 text-slate-400" />
                      <label className="font-medium text-slate-700">Weight (kg)</label>
                    </div>
                    <Input
                      type="number"
                      placeholder="70"
                      value={profile.weight}
                      onChange={(e) => setProfile(p => ({ ...p, weight: e.target.value }))}
                      className="border-0 text-xl font-semibold p-0 h-auto focus-visible:ring-0"
                    />
                  </div>
                  <div className="bg-white rounded-2xl p-4 border border-slate-200">
                    <div className="flex items-center gap-3 mb-2">
                      <Ruler className="w-5 h-5 text-slate-400" />
                      <label className="font-medium text-slate-700">Height (cm)</label>
                    </div>
                    <Input
                      type="number"
                      placeholder="170"
                      value={profile.height}
                      onChange={(e) => setProfile(p => ({ ...p, height: e.target.value }))}
                      className="border-0 text-xl font-semibold p-0 h-auto focus-visible:ring-0"
                    />
                  </div>
                  <div className="bg-white rounded-2xl p-4 border border-slate-200">
                    <div className="flex items-center gap-3 mb-2">
                      <Target className="w-5 h-5 text-slate-400" />
                      <label className="font-medium text-slate-700">Age</label>
                    </div>
                    <Input
                      type="number"
                      placeholder="30"
                      value={profile.age}
                      onChange={(e) => setProfile(p => ({ ...p, age: e.target.value }))}
                      className="border-0 text-xl font-semibold p-0 h-auto focus-visible:ring-0"
                    />
                  </div>
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 3 && (
            <SlideTransition key="activity">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Activity level</h1>
                  <p className="text-slate-500 mt-1">How active are you?</p>
                </div>
                <div className="space-y-3">
                  {activityLevels.map(level => (
                    <motion.button
                      key={level.id}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile(p => ({ ...p, activity_level: level.id }))}
                      className={`w-full p-4 rounded-2xl border-2 text-left transition-all ${
                        profile.activity_level === level.id
                          ? 'border-emerald-500 bg-emerald-50'
                          : 'border-slate-200 bg-white hover:border-slate-300'
                      }`}
                    >
                      <div className="flex items-center justify-between">
                        <div>
                          <p className="font-semibold text-slate-800">{level.label}</p>
                          <p className="text-sm text-slate-500">{level.desc}</p>
                        </div>
                        {profile.activity_level === level.id && (
                          <Check className="w-5 h-5 text-emerald-500" />
                        )}
                      </div>
                    </motion.button>
                  ))}
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 4 && (
            <SlideTransition key="allergies">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Any allergies?</h1>
                  <p className="text-slate-500 mt-1">We'll alert you when scanning</p>
                </div>
                <div className="flex flex-wrap gap-2">
                  {commonAllergens.map(allergen => (
                    <Badge
                      key={allergen}
                      variant={profile.allergies.includes(allergen) ? 'default' : 'outline'}
                      className={`px-4 py-2 text-sm cursor-pointer transition-all ${
                        profile.allergies.includes(allergen)
                          ? 'bg-red-500 hover:bg-red-600 border-red-500'
                          : 'hover:border-slate-400'
                      }`}
                      onClick={() => toggleAllergen(allergen)}
                    >
                      {profile.allergies.includes(allergen) && (
                        <AlertCircle className="w-3 h-3 mr-1" />
                      )}
                      {allergen}
                    </Badge>
                  ))}
                </div>
                <p className="text-sm text-slate-400">Tap to select, skip if none</p>
              </div>
            </SlideTransition>
          )}
        </AnimatePresence>
      </div>

      {/* Navigation */}
      <div className="p-6 bg-white border-t border-slate-100">
        <div className="flex gap-3">
          {step > 0 && (
            <Button
              variant="outline"
              onClick={() => setStep(s => s - 1)}
              className="h-14 px-6 rounded-2xl"
            >
              <ChevronLeft className="w-5 h-5" />
            </Button>
          )}
          <Button
            onClick={() => step < totalSteps - 1 ? setStep(s => s + 1) : handleComplete()}
            disabled={!canProceed()}
            className="flex-1 h-14 rounded-2xl text-base bg-gradient-to-r from-emerald-500 to-teal-600 hover:from-emerald-600 hover:to-teal-700"
          >
            {step < totalSteps - 1 ? (
              <>
                Continue
                <ChevronRight className="w-5 h-5 ml-2" />
              </>
            ) : (
              <>
                <Sparkles className="w-5 h-5 mr-2" />
                Start Scanning
              </>
            )}
          </Button>
        </div>
      </div>
    </div>
  );
}

# ---- merged ----

{
  "name": "FoodScan",
  "type": "object",
  "properties": {
    "food_name": {
      "type": "string",
      "description": "Identified food name"
    },
    "image_url": {
      "type": "string",
      "description": "URL of the scanned food image"
    },
    "calories": {
      "type": "number",
      "description": "Estimated calories"
    },
    "protein": {
      "type": "number",
      "description": "Protein in grams"
    },
    "carbs": {
      "type": "number",
      "description": "Carbohydrates in grams"
    },
    "fats": {
      "type": "number",
      "description": "Fats in grams"
    },
    "fiber": {
      "type": "number",
      "description": "Fiber in grams"
    },
    "sugar": {
      "type": "number",
      "description": "Sugar in grams"
    },
    "sodium": {
      "type": "number",
      "description": "Sodium in mg"
    },
    "portion_size": {
      "type": "string",
      "description": "Estimated portion size"
    },
    "allergens": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "List of allergens"
    },
    "health_score": {
      "type": "number",
      "description": "Health score 1-10"
    },
    "meal_type": {
      "type": "string",
      "enum": [
        "breakfast",
        "lunch",
        "dinner",
        "snack"
      ],
      "description": "Type of meal"
    },
    "tags": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Custom tags for categorization"
    },
    "scan_date": {
      "type": "string",
      "format": "date",
      "description": "Date of scan"
    },
    "suggestions": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "AI suggestions for healthier choices"
    },
    "food_category": {
      "type": "string",
      "description": "Category like fruit, vegetable, protein, etc."
    }
  },
  "required": [
    "food_name",
    "calories"
  ]
}

# ---- merged ----

import React from 'react';
import { Link } from 'react-router-dom';
import { createPageUrl } from '@/utils';
import { Home, Camera, Clock, User } from 'lucide-react';
import { motion } from 'framer-motion';

const navItems = [
  { name: 'Dashboard', icon: Home, label: 'Home' },
  { name: 'Scan', icon: Camera, label: 'Scan' },
  { name: 'History', icon: Clock, label: 'History' },
  { name: 'Profile', icon: User, label: 'Profile' },
];

export default function Layout({ children, currentPageName }) {
  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30">
      <style>{`
        :root {
          --color-primary: #10b981;
          --color-primary-dark: #059669;
        }
        
        * {
          -webkit-tap-highlight-color: transparent;
        }
        
        .nav-active {
          background: linear-gradient(135deg, #10b981 0%, #14b8a6 100%);
        }
      `}</style>
      
      <main className="pb-24">
        {children}
      </main>

      {/* Bottom Navigation */}
      <nav className="fixed bottom-0 inset-x-0 bg-white/80 backdrop-blur-xl border-t border-slate-200/50 z-50 safe-area-bottom">
        <div className="max-w-lg mx-auto px-4">
          <div className="flex items-center justify-around h-16">
            {navItems.map((item) => {
              const isActive = currentPageName === item.name;
              const Icon = item.icon;
              
              return (
                <Link
                  key={item.name}
                  to={createPageUrl(item.name)}
                  className="relative flex flex-col items-center justify-center w-16 h-full"
                >
                  <motion.div
                    whileTap={{ scale: 0.9 }}
                    className={`flex flex-col items-center justify-center ${
                      isActive ? 'text-emerald-600' : 'text-slate-400'
                    }`}
                  >
                    {item.name === 'Scan' ? (
                      <div className="absolute -top-6">
                        <div className="w-14 h-14 rounded-full nav-active flex items-center justify-center shadow-lg shadow-emerald-500/30">
                          <Icon className="w-6 h-6 text-white" />
                        </div>
                      </div>
                    ) : (
                      <>
                        <Icon className={`w-5 h-5 ${isActive ? 'stroke-[2.5px]' : ''}`} />
                        <span className={`text-xs mt-1 ${isActive ? 'font-medium' : ''}`}>
                          {item.label}
                        </span>
                      </>
                    )}
                  </motion.div>
                  
                  {isActive && item.name !== 'Scan' && (
                    <motion.div
                      layoutId="activeTab"
                      className="absolute -top-0.5 w-8 h-1 bg-gradient-to-r from-emerald-500 to-teal-500 rounded-full"
                    />
                  )}
                </Link>
              );
            })}
          </div>
        </div>
      </nav>
    </div>
  );
}

# ---- merged ----

import React, { useState, useEffect } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery, useMutation, useQueryClient } from '@tanstack/react-query';
import { motion, AnimatePresence } from 'framer-motion';
import { format } from 'date-fns';
import { toast } from 'sonner';
import { ArrowLeft, Utensils, Tag, X } from 'lucide-react';
import { Button } from "@/components/ui/button";
import {
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
} from "@/components/ui/select";
import { Badge } from "@/components/ui/badge";

import CameraScanner from '../components/scanner/CameraScanner';
import ScanResult from '../components/scanner/ScanResult';
import OnboardingFlow from '../components/onboarding/OnboardingFlow';

export default function ScanPage() {
  const [isProcessing, setIsProcessing] = useState(false);
  const [scanResult, setScanResult] = useState(null);
  const [mealType, setMealType] = useState('snack');
  const [selectedTags, setSelectedTags] = useState([]);
  const [uploadedImageUrl, setUploadedImageUrl] = useState(null);

  const availableTags = [
    'Homemade', 'Restaurant', 'Takeout', 'Meal Prep',
    'Cheat Meal', 'Pre-Workout', 'Post-Workout', 'Late Night',
    'Comfort Food', 'Healthy Choice', 'Organic', 'Fast Food'
  ];
  
  const queryClient = useQueryClient();

  const { data: userProfiles, isLoading: profileLoading } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles?.[0];
  const needsOnboarding = !profileLoading && (!userProfile || !userProfile.onboarding_complete);

  const createProfileMutation = useMutation({
    mutationFn: (data) => base44.entities.UserProfile.create(data),
    onSuccess: () => {
      queryClient.invalidateQueries({ queryKey: ['userProfile'] });
    },
  });

  const createScanMutation = useMutation({
    mutationFn: (data) => base44.entities.FoodScan.create(data),
    onSuccess: () => {
      queryClient.invalidateQueries({ queryKey: ['foodScans'] });
      toast.success('Meal added to your log!');
      setScanResult(null);
      setUploadedImageUrl(null);
    },
  });

  const handleOnboardingComplete = async (profileData) => {
    await createProfileMutation.mutateAsync(profileData);
  };

  const handleCapture = async (file) => {
    setIsProcessing(true);
    setScanResult(null);

    try {
      // Upload image first
      const { file_url } = await base44.integrations.Core.UploadFile({ file });
      setUploadedImageUrl(file_url);

      // Analyze with AI
      const analysisResult = await base44.integrations.Core.InvokeLLM({
        prompt: `Analyze this food image and provide detailed nutritional information.
        
User profile:
- Dietary goal: ${userProfile?.dietary_goal || 'general health'}
- Diet type: ${userProfile?.diet_type || 'none'}
- Allergies: ${userProfile?.allergies?.join(', ') || 'none'}

Please identify the food and provide:
1. Food name and category
2. Estimated portion size
3. Calories and macronutrients (protein, carbs, fats, fiber, sugar, sodium)
4. Health score (1-10) based on the user's dietary goal
5. List of potential allergens
6. 2-3 personalized suggestions for healthier choices or tips

Be specific and accurate with nutritional estimates based on typical serving sizes.`,
        file_urls: [file_url],
        response_json_schema: {
          type: 'object',
          properties: {
            food_name: { type: 'string' },
            food_category: { type: 'string' },
            portion_size: { type: 'string' },
            calories: { type: 'number' },
            protein: { type: 'number' },
            carbs: { type: 'number' },
            fats: { type: 'number' },
            fiber: { type: 'number' },
            sugar: { type: 'number' },
            sodium: { type: 'number' },
            health_score: { type: 'number' },
            allergens: { type: 'array', items: { type: 'string' } },
            suggestions: { type: 'array', items: { type: 'string' } },
          },
          required: ['food_name', 'calories', 'protein', 'carbs', 'fats']
        }
      });

      setScanResult({
        ...analysisResult,
        image_url: file_url,
      });
    } catch (error) {
      console.error('Scan failed:', error);
      toast.error('Failed to analyze food. Please try again.');
    } finally {
      setIsProcessing(false);
    }
  };

  const handleAddToLog = async () => {
    if (!scanResult) return;

    await createScanMutation.mutateAsync({
      ...scanResult,
      meal_type: mealType,
      tags: selectedTags,
      scan_date: format(new Date(), 'yyyy-MM-dd'),
    });
  };

  const handleScanAgain = () => {
    setScanResult(null);
    setUploadedImageUrl(null);
    setSelectedTags([]);
  };

  const toggleTag = (tag) => {
    setSelectedTags(prev => 
      prev.includes(tag) 
        ? prev.filter(t => t !== tag)
        : [...prev, tag]
    );
  };

  if (profileLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  if (needsOnboarding) {
    return <OnboardingFlow onComplete={handleOnboardingComplete} />;
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30">
      <div className="max-w-lg mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="flex items-center justify-between mb-8"
        >
          <div>
            <h1 className="text-2xl font-bold text-slate-800">Scan Food</h1>
            <p className="text-slate-500 text-sm">Point your camera at any meal</p>
          </div>
          {scanResult && (
            <Button
              variant="ghost"
              size="icon"
              onClick={handleScanAgain}
              className="rounded-full"
            >
              <ArrowLeft className="w-5 h-5" />
            </Button>
          )}
        </motion.div>

        {/* Meal Type Selector */}
        {!scanResult && (
          <motion.div 
            initial={{ opacity: 0 }}
            animate={{ opacity: 1 }}
            className="space-y-4 mb-6"
          >
            <Select value={mealType} onValueChange={setMealType}>
              <SelectTrigger className="w-full h-12 rounded-2xl bg-white border-slate-200">
                <div className="flex items-center gap-2">
                  <Utensils className="w-4 h-4 text-slate-400" />
                  <SelectValue placeholder="Meal type" />
                </div>
              </SelectTrigger>
              <SelectContent>
                <SelectItem value="breakfast">🌅 Breakfast</SelectItem>
                <SelectItem value="lunch">☀️ Lunch</SelectItem>
                <SelectItem value="dinner">🌙 Dinner</SelectItem>
                <SelectItem value="snack">🍎 Snack</SelectItem>
              </SelectContent>
            </Select>

            {/* Tags Selector */}
            <div className="bg-white rounded-2xl p-4 border border-slate-200">
              <div className="flex items-center gap-2 mb-3">
                <Tag className="w-4 h-4 text-slate-400" />
                <span className="text-sm font-medium text-slate-600">Add Tags (Optional)</span>
              </div>
              <div className="flex flex-wrap gap-2">
                {availableTags.map(tag => {
                  const isSelected = selectedTags.includes(tag);
                  return (
                    <Badge
                      key={tag}
                      variant={isSelected ? "default" : "outline"}
                      className={`cursor-pointer transition-all ${
                        isSelected 
                          ? 'bg-emerald-500 hover:bg-emerald-600 text-white' 
                          : 'hover:border-emerald-300'
                      }`}
                      onClick={() => toggleTag(tag)}
                    >
                      {tag}
                      {isSelected && <X className="w-3 h-3 ml-1" />}
                    </Badge>
                  );
                })}
              </div>
            </div>
          </motion.div>
        )}

        {/* Scanner or Result */}
        <AnimatePresence mode="wait">
          {scanResult ? (
            <motion.div
              key="result"
              initial={{ opacity: 0, scale: 0.95 }}
              animate={{ opacity: 1, scale: 1 }}
              exit={{ opacity: 0, scale: 0.95 }}
            >
              <ScanResult
                result={scanResult}
                onAddToLog={handleAddToLog}
                onScanAgain={handleScanAgain}
                userAllergies={userProfile?.allergies || []}
              />
            </motion.div>
          ) : (
            <motion.div
              key="scanner"
              initial={{ opacity: 0 }}
              animate={{ opacity: 1 }}
              exit={{ opacity: 0 }}
              className="bg-white rounded-3xl p-8 shadow-sm border border-slate-100"
            >
              <CameraScanner
                onCapture={handleCapture}
                isProcessing={isProcessing}
              />
            </motion.div>
          )}
        </AnimatePresence>

        {/* Tips */}
        {!scanResult && !isProcessing && (
          <motion.div 
            initial={{ opacity: 0, y: 20 }}
            animate={{ opacity: 1, y: 0 }}
            transition={{ delay: 0.3 }}
            className="mt-8 space-y-3"
          >
            <p className="text-sm font-medium text-slate-600 mb-3">📸 Scanning tips</p>
            <div className="grid gap-2 text-sm text-slate-500">
              <div className="flex items-start gap-2">
                <span className="w-5 h-5 rounded-full bg-emerald-100 text-emerald-600 flex items-center justify-center text-xs flex-shrink-0">1</span>
                <span>Ensure good lighting for accurate results</span>
              </div>
              <div className="flex items-start gap-2">
                <span className="w-5 h-5 rounded-full bg-emerald-100 text-emerald-600 flex items-center justify-center text-xs flex-shrink-0">2</span>
                <span>Capture the entire plate or food item</span>
              </div>
              <div className="flex items-start gap-2">
                <span className="w-5 h-5 rounded-full bg-emerald-100 text-emerald-600 flex items-center justify-center text-xs flex-shrink-0">3</span>
                <span>Works with meals, snacks, and packaged foods</span>
              </div>
            </div>
          </motion.div>
        )}
      </div>
    </div>
  );
}

# ---- merged ----

# merged script
print('Hello')